<a href="https://colab.research.google.com/github/arfaluvess/automation_project/blob/main/OOS_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING NECESSARY LIBRARY**


In [ ]:
# @title Import any necessary libraries|modules
%matplotlib inline

# Data Manipulation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import pandas as pd
import polars as pl
import numpy as np
import os
import glob
import win32com.client as win32
import tempfile
import openpyxl
import re
from datetime import datetime, timedelta, date
from tabulate import tabulate

# Data Visualization if needed
import plotly.io as pio
pio.renderers.default = 'iframe'
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Declare local username
username = os.path.expanduser('~')
print(username)

C:\Users\arfahanis


In [ ]:
# @title Declare weeks and date format

# Get the current date and the corresponding Monday
today = datetime.now()
current_week_number = today.isocalendar()[1]
current_year = today.year

# Function to calculate week number considering wrap around of weeks at year end
def get_past_week_number(weeks_ago):
    past_week_number = current_week_number - weeks_ago
    if past_week_number < 1:
        past_week_number += 52  # Assuming a standard year with 52 weeks
    return past_week_number

# Generate past week numbers
one_week_ago = get_past_week_number(1)
two_week_ago = get_past_week_number(2)
three_week_ago = get_past_week_number(3)
four_week_ago = get_past_week_number(4)

# Calculate the corresponding Monday dates for each week
# Use timedelta directly instead of datetime.timedelta
monday_of_week = today - timedelta(days=today.weekday())  # This week's Monday
monday_1weekAgo = monday_of_week - timedelta(weeks=1)
monday_2weekAgo = monday_of_week - timedelta(weeks=2)
monday_3weekAgo = monday_of_week - timedelta(weeks=3)
monday_4weekAgo = monday_of_week - timedelta(weeks=4)

# Format for folder names
week_folder_pattern = f"WK{current_week_number}"
monday_str = monday_of_week.strftime('%Y-%m-%d')
monday_1week = monday_1weekAgo.strftime('%Y-%m-%d')
monday_2week = monday_2weekAgo.strftime('%Y-%m-%d')
monday_3week = monday_3weekAgo.strftime('%Y-%m-%d')
monday_4week = monday_4weekAgo.strftime('%Y-%m-%d')

# Output to verify
print(f"Current Week Number: {current_week_number}")
print(f"Current Year: {current_year}")
print(f"One week ago: WK{one_week_ago}")
print(f"Two week ago: WK{two_week_ago}")
print(f"Three week ago: WK{three_week_ago}")
print(f"Four week ago: WK{four_week_ago}")
print(f"Monday of the week: {monday_str}")
print(f"Monday last 1 week: {monday_1week}")
print(f"Monday last 2 weeks: {monday_2week}")
print(f"Monday last 3 weeks: {monday_3week}")
print(f"Monday last 4 weeks: {monday_4week}")
print(f"This week folder: {week_folder_pattern}")

Current Week Number: 2
Current Year: 2025
One week ago: WK1
Two week ago: WK52
Three week ago: WK51
Four week ago: WK50
Monday of the week: 2025-01-06
Monday last 1 week: 2024-12-30
Monday last 2 weeks: 2024-12-23
Monday last 3 weeks: 2024-12-16
Monday last 4 weeks: 2024-12-09
This week folder: WK2


## DATA LOADING / GET DATA

---

* Item Master
* Warehouse SOH
* Average daily sales (ADS)
* Purchase 02 Food & Non Food
* Service level (8 Weeks)
* Fulfillment
* Ship out vs demand
* Adjustment





## INTERNAL ITEM MASTER & BC ITEM MASTER

In [ ]:
# Load item master, item no as pl.utf or string
itemmaster_directory = fr"{username}\OneDrive - Eco-Shop\Inventory\Share\MinMax\Internal\Item_Master\*.csv"
itemmaster_df = pl.scan_csv(itemmaster_directory,
                             infer_schema_length=100000,
                            ).collect()


# Load the BC Item master
bc_directory = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Ordering File\Master File\BC - Item Master\RunReport - BC Item Master.xlsm"
bc_df = pd.read_excel(bc_directory, sheet_name="Data",
                              usecols=['Item No','ABC SKUs','Vendor Code','Vendor Name'],
                              dtype={'Item No':str})
# Convert to Polars DataFrame
bc_df = pl.from_pandas(bc_df)

# Rename 'Item No' to 'ItemNo' in bc_df for consistency in merging
bc_df = bc_df.rename({"Item No": "ItemNo"})

# Drop duplicates of ItemNo from bc_df and leave first unique occurences
bc_df = bc_df.unique(subset=["ItemNo"])

## MINMAX FILES

In [ ]:
# Retrieve latest or latest 4 weeks minmax files (monday's file of every last 4 weeks ago)
minmax_directory = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Min Max"

# Find the latest file by time
latestMinmax = max(glob.glob(minmax_directory + "/*.csv"), key=os.path.getctime)
print(f"The latest minmax file: {latestMinmax}")

# Find latest minmax file, by last week (before new week/ monday)
minmax1_df = pl.scan_csv(latestMinmax,
                        infer_schema_length=100000,
                    ).collect()

# Read all past monday of the file then merged monday's minmax file for 4 weeks
mon_files = [monday_1week,monday_2week,monday_3week,monday_4week]
week_files = [one_week_ago,two_week_ago,three_week_ago,four_week_ago]
minmax_dfs = []

for minmax_file, week_number in zip(mon_files, week_files):
    mx_pattern = os.path.join(minmax_directory, f"GEDQEstimate - {minmax_file}*.csv")
    mx_files = glob.glob(mx_pattern, recursive=True)
    for mx_file in mx_files:
        print(f"Processing minmax file: {mx_file}")
        minmax_df = pl.scan_csv(mx_file, infer_schema_length=100000).collect()

        # Standardizing columns types
        minmax_df = minmax_df.with_columns(
            pl.col('week1triggersoft').cast(pl.Int64),
            pl.col('WJM_UOM_qty').cast(pl.Int64),
            pl.col('WKL_UOM_qty').cast(pl.Int64),
            pl.col('ItemNo').cast(pl.Utf8)
        )

        # Add columns to all dataframe that indicate their week based on their file name or date in their excel name
        minmax_df = minmax_df.with_columns(pl.lit(week_number).alias('Week'))

        # Remove unnecessary columns
        columns_to_drop = [col for col in minmax_df.columns if 'triggersoft' in col and col != 'week1triggersoft']
        minmax_df = minmax_df.drop(columns_to_drop)

        minmax_dfs.append(minmax_df)

# Combine all those dataframe into single dataframe, using vstack or any other methods
minmax4weeks = pl.concat(minmax_dfs,how='vertical')

# Convert 'week1triggersoft' column to int64 and ItemNo as str
minmax4weeks = minmax4weeks.with_columns(
    pl.col('week1triggersoft').cast(pl.Int64),  # Converts to int64
    pl.col('ItemNo').cast(pl.Utf8)
)

# Reorder Week column to the first column loc
minmax4weeks = minmax4weeks.select(['Week'] + [col for col in minmax4weeks.columns if col != 'Week'])

# Only select fews columns to be chosen and use
minmax4weeks = minmax4weeks.select(['ItemNo','Week','week1triggersoft'])

The latest minmax file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Min Max\GEDQEstimate - 2025-01-06.csv
Processing minmax file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Min Max\GEDQEstimate - 2024-12-30.csv
Processing minmax file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Min Max\GEDQEstimate - 2024-12-23.csv
Processing minmax file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Min Max\GEDQEstimate - 2024-12-16.csv
Processing minmax file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Min Max\GEDQEstimate - 2024-12-09.csv


## PURCHASE02 - ORDER SALES QLIKSENSES
### INVENTORY05 - AVERAGE DAILY SALES & RL98 SALES

In [ ]:
# Read latest purchasing order sales pur02 Qliksense to retrieve Last receive date
Pur02_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Pur02Sales"
latestPur02 = max(glob.glob(Pur02_path + "/*.xlsx"), key=os.path.getctime)
print(f"The latest Pur02 file: {latestPur02}")

# Read the file using polars
pur02_df = (
    pl.read_excel(
        latestPur02,
        infer_schema_length=400000,
        read_options={'columns': ['Item No', 'Last RC Date',"1st TR In Date"]},
        schema_overrides={'Item No': pl.Utf8}
    )
    .filter(pl.col('Item No') != 'Totals')  # Filter out rows with 'Totals' in Item No
    .rename({'Item No': 'ItemNo'})  # Rename 'Item No' to 'ItemNo'
    .with_columns([
    pl.col('Last RC Date').str.strptime(pl.Date, '%d/%m/%Y', strict=False),
    pl.col('1st TR In Date').str.strptime(pl.Date, '%d/%m/%Y', strict=False)
])
)

# Read latest inventory average daily sales
ADS_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Inv05ADS"
latestADS = max(glob.glob(ADS_path + "/*.xlsx"), key=os.path.getctime)
print(f"The latest ADS file: {latestADS}")

# Read the file using polars
ADS_df = (
    pl.read_excel(
        latestADS,
        infer_schema_length=100000,
        schema_overrides={'Item No': pl.Utf8}
    )
    .filter(pl.col('Item No') != 'Totals')  # Filter out rows with 'Totals' in Item No
    .rename({'Item No': 'ItemNo'})  # Rename 'Item No' to 'ItemNo'
    .with_columns([
        pl.col("ADS Qty").round(0).cast(pl.Int64),  # Round and cast to integer
        pl.col("RL91 Sales Qty").round(0).cast(pl.Int64)  # Round and cast to integer
    ])
)

The latest Pur02 file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Pur02Sales\20240106.xlsx
The latest ADS file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Inv05ADS\20240106.xlsx


In [ ]:
# @title WAREHOUSE CHECK YW

wh_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\WarehouseYW"

# Find the latest Excel file in the specified directory
latestWH = max(glob.glob(os.path.join(wh_path, "*.xlsx")), key=os.path.getctime)
print(f"Processing the latest file: {latestWH}")

# Read specific sheets using pandas with the specified header row
df_es_wji_oos_pd = pd.read_excel(latestWH, sheet_name='ES WJI OOS', header=4)
df_es_wkl_oos_pd = pd.read_excel(latestWH, sheet_name='ES WKL OOS', header=4)
df_eco_plus_oos_pd = pd.read_excel(latestWH, sheet_name='ECO PLUS OOS', header=4)

# Convert the pandas DataFrames to polars DataFrames
df_es_wji_oos = pl.from_pandas(df_es_wji_oos_pd).rename({"SKU": "ItemNo"}).with_columns(
    pl.col("ItemNo").cast(pl.Int64),
    pl.col("断货天数 (No. of Days)").cast(pl.Int64)
)
df_es_wkl_oos = pl.from_pandas(df_es_wkl_oos_pd).rename({"SKU": "ItemNo"}).with_columns(
    pl.col("ItemNo").cast(pl.Int64),
    pl.col("断货天数 (No. of Days)").cast(pl.Int64)
)
df_eco_plus_oos = pl.from_pandas(df_eco_plus_oos_pd).rename({"SKU": "ItemNo"}).with_columns(
    pl.col("ItemNo").cast(pl.Int64),
    pl.col("断货天数 (No. of Days)").cast(pl.Int64)
)

WHOOS_yw = pl.concat([
    df_es_wji_oos.select(["ItemNo", "Item Name", "断货天数 (No. of Days)"]),
    df_es_wkl_oos.select(["ItemNo", "Item Name", "断货天数 (No. of Days)"]),
    df_eco_plus_oos.select(["ItemNo", "Item Name", "断货天数 (No. of Days)"])
], how='vertical')

display(WHOOS_yw)

Processing the latest file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\WarehouseYW\Week2.xlsx


ItemNo,Item Name,断货天数 (No. of Days)
i64,str,i64
10010263,"""MAYORA TAMARIND CANDY 135G""",0
10010347,"""KOPIKO CAPPUCINO CANDY 120G""",0
10010348,"""KOPIKO COFFEE CANDY 120G""",2
10010386,"""FOX'S FRUITY MINTS OVAL CANDY …",0
10010389,"""FOX'S ASSORTED COFFEE WORLD OV…",0
…,…,…
80367011,"""EP INERTIA CARPET SET ASST 130…",4
80387016,"""ECO-P DINOSAUR PLASTER ASST 5.…",0
80387017,"""ECO-P ANIMAL PLASTER ASST 5CMX…",0


In [ ]:
# Convert ItemNo from int to str, then only choose rows of ItemNo that 断货天数 (No. of Days) > 0
WHOOS_yw = WHOOS_yw.with_columns(
    pl.col('ItemNo').cast(pl.Utf8)
)
WHOOS_yw = WHOOS_yw.filter(pl.col('断货天数 (No. of Days)') > 0)
display(WHOOS_yw)

ItemNo,Item Name,断货天数 (No. of Days)
str,str,i64
"""10010348""","""KOPIKO COFFEE CANDY 120G""",2
"""10015001""","""TORRONE CANDY COOL MINT 120G""",2
"""10015009""","""TORRONE CHOC MINT SWEET 120G""",4
"""10015080""","""JOMEI MINI CC STICK 25S 100G""",7
"""10015214""","""RINDA CANDY ASSORTED FRUITS FL…",2
…,…,…
"""80277078""","""ECO-P SLIDE EJECTION RAIL CAR …",4
"""80337009""","""ECO-P TOYS TOOL ASST 888-1-5 4…",7
"""80347008""","""ELECTRIC FISHING ASSORTED 22.5…",7


In [ ]:
# Save WHOOS_yw to excel
WHOOS_yw.write_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\Test\WHOOS_yw.xlsx")

## WAREHOUSE SOH

In [ ]:
# Retrieve latest file for laetst warehouse SOH
whsoh_directory = fr"{username}\OneDrive - Eco-Shop\Inventory\Share\MinMax\Internal\WH_Master\*.csv"
latestWHSOH = max(glob.glob(whsoh_directory), key=os.path.getctime)
print(f"The latest WHSOH file: {latestWHSOH}")

SOH_df = pl.scan_csv(latestWHSOH,
                    infer_schema_length=100000,
                    schema_overrides={'itcode': pl.Utf8}
                    ).collect()

# Rename itcode to ItemNo
SOH_df = SOH_df.rename({'itcode':'ItemNo',
                        'wareid':'WarehouseID',
                        'available_qty':'AvailableQty',
                        'reserve_qty':'ReserveQty',
                        'Wh_available_qty':'WHAvailable_Qty'})

# Drop columns: date	created_at	created_by	updated_at	updated_by	imported_at	imported_by
SOH_df = SOH_df.drop(['key','date','created_at','created_by','updated_at','updated_by','imported_at','imported_by','createDate'])

# Add a new 'WarehouseSOH' column based on 'WarehouseID'
SOH_df = SOH_df.with_columns(
    pl.when(pl.col('WarehouseID') == 'WKL').then(pl.col('AvailableQty'))
      .when(pl.col('WarehouseID') == 'WJI').then(pl.col('WHAvailable_Qty'))
      .otherwise(None).alias('WarehouseSOH')
)

# Convert all float columns to integers
cols = ['AvailableQty',	'ReserveQty',	'WHAvailable_Qty', 'WarehouseSOH']
SOH_df = SOH_df.with_columns([pl.col(col).cast(pl.Int64) for col in cols])

display(SOH_df)

The latest WHSOH file: C:\Users\arfahanis\OneDrive - Eco-Shop\Inventory\Share\MinMax\Internal\WH_Master\WhSoh_2025-01-12.csv


WarehouseID,ItemNo,AvailableQty,ReserveQty,WHAvailable_Qty,WarehouseSOH
str,str,i64,i64,i64,i64
"""WJI""","""80050404""",7968,1656,6312,6312
"""WJI""","""80050405""",960,1104,-144,-144
"""WJI""","""80050406""",19080,1704,17376,17376
"""WJI""","""80050408""",6168,720,5448,5448
"""WJI""","""80050409""",144,192,-48,-48
…,…,…,…,…,…
"""WJI""","""99020056""",6,0,6,6
"""WJI""","""99090204""",2,0,2,2
"""WJI""","""99060078""",343,0,343,343


In [ ]:
# Drop other columns to leave warehouseSOH using polars
SOH_df = SOH_df.select(['ItemNo','WarehouseID','WarehouseSOH'])
display(SOH_df)

ItemNo,WarehouseID,WarehouseSOH
str,str,i64
"""80050404""","""WJI""",6312
"""80050405""","""WJI""",-144
"""80050406""","""WJI""",17376
"""80050408""","""WJI""",5448
"""80050409""","""WJI""",-48
…,…,…
"""99020056""","""WJI""",6
"""99090204""","""WJI""",2
"""99060078""","""WJI""",343


In [ ]:
# Merge WH SOH with Item master and BC master, only select few columns to combine
itemmaster_cols = [
    'ItemNo', 'ItemName','Division', 'Department', 'Category', 'HouseBrandIndicator',
    'RawMaterialIndicator', 'StockReorderIndicator', 'BuyerGroup', 'Local/Oversea',
    'ProductEntity', 'RQStatus'
]

# Make sure 'ItemNo' is included in the columns to allow for merging
itemmaster_df = itemmaster_df[itemmaster_cols]

# Merge itemmaster_df_selected with SOH_df on 'ItemNo'
WH_soh = SOH_df.join(itemmaster_df, on='ItemNo', how='left')

# Perform the join, keeping only the 'ABC SKUs' column from bc_df
WH_soh = WH_soh.join(bc_df.select(['ItemNo','ABC SKUs','Vendor Code','Vendor Name']), on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_38320\2989631055.py:12: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_38320\2989631055.py:15: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Define the conditions for the new column 'Item Maintenance'
WH_soh = WH_soh.with_columns(
    pl.when((pl.col('ABC SKUs') == "Z") & (pl.col('ProductEntity') == "Eco-Shop"))
      .then(pl.lit("New Item + Eco-Shop"))  # Set value as "New Item + Eco-Shop" if conditions match
      .when((pl.col('ABC SKUs') == "Z") & (pl.col('ProductEntity') == "Eco-Plus"))
      .then(pl.lit("New Item + Eco Plus Resume Order"))  # Set value as "New Item + Eco Plus continue order" if conditions match
      .when((pl.col('ABC SKUs') != "Z") & (pl.col('ProductEntity') == "Eco-Plus"))
      .then(pl.lit("EcoPlus Item continue order WK47"))
      .otherwise(pl.lit(""))
      .alias("Item Maintenance")
)

In [ ]:
# Combine pur02_df & ADS_df into WH_soh
WH_soh = WH_soh.join(pur02_df, on='ItemNo', how='left').join(ADS_df, on='ItemNo', how='left')

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_38320\1880069609.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Step 1: Calculate 'TR date New Item' first
WH_soh = WH_soh.with_columns(
    pl.when((pl.col('ABC SKUs') == 'Z') & (pl.col('1st TR In Date').is_not_null()))
      .then((pl.lit(today) - pl.col('1st TR In Date')).dt.total_days())
      .otherwise(None)
      .alias('TR date New Item')
)

# Step 2: Calculate 'ADS Qty New Item' using the previously created 'TR date New Item'
WH_soh = WH_soh.with_columns(
    pl.when(pl.col('ABC SKUs') == 'Z')
      .then(pl.col('RL91 Sales Qty') / pl.col('TR date New Item').fill_null(1))  # Avoid division by zero
      .otherwise(pl.col('ADS Qty'))
      .alias('ADS Qty New Item')
)

# Step 3: Calculate 'stock_cover_days' based on 'WarehouseSOH' and 'ADS Qty'
WH_soh = WH_soh.with_columns(
    pl.when(pl.col('ABC SKUs') == 'Z')
      .then(pl.col('WarehouseSOH') / pl.col('ADS Qty New Item').fill_null(1))
      .otherwise(pl.col('WarehouseSOH') / pl.col('RL91 Sales Qty').fill_null(1))
      .alias('Stock cover days')
)

In [ ]:
# Define conditions based on ProductEntity, WarehouseID, and stock_cover_days

# Step 1: Define SOH conditions
WH_soh = WH_soh.with_columns(
    pl.when((pl.col("ProductEntity") == "Eco-Shop") &
            (pl.col("WarehouseID").is_in(["WJI", "WKL"])) &
            (pl.col("WarehouseSOH") < 500))
      .then(True)
      .when((pl.col("ProductEntity") == "Eco-Shop") &
            (pl.col("WarehouseID") == "2WH") &
            (pl.col("WarehouseSOH") < 300))
      .then(True)
      .when((pl.col("ProductEntity") == "Eco-Plus") &
            (pl.col("WarehouseID").is_in(["WJI", "WKL"])) &
            (pl.col("WarehouseSOH") < 10))
      .then(True)
      .when((pl.col("ProductEntity") == "Eco-Plus") &
            (pl.col("WarehouseID") == "2WH") &
            (pl.col("WarehouseSOH") < 10))
      .then(True)
      .otherwise(False)
      .alias("SOH_Condition")
)

# Step 2: Calculate 'StockCoverDays_Condition' based on Local/Oversea and rounded 'Stock cover days'
WH_soh = WH_soh.with_columns(
    pl.when((pl.col("Local/Oversea") == "LOCAL") & (pl.col("Stock cover days").round(3) <= 3))
      .then(True)
      .when((pl.col("Local/Oversea") == "OVERSEA") & (pl.col("Stock cover days").round(3) <= 3))
      .then(True)
      .otherwise(False)
      .alias("StockCoverDays_Condition")
)

# Step 3: Define the final OOS condition based on the conditions from Steps 1 and 2, using `pl.lit`
WH_soh = WH_soh.with_columns(
    pl.when((pl.col("SOH_Condition") == True) & (pl.col("StockCoverDays_Condition") == True))
      .then(pl.lit("OOS"))
      .otherwise(pl.lit("In Stock"))
      .alias("OOS_Status")
)

In [ ]:
# Check Z item and the column OOS_Status is OOS
WH_soh.filter((pl.col('ABC SKUs') == 'Z') & (pl.col("OOS_Status") == "OOS")).sample(8, with_replacement=True)

ItemNo,WarehouseID,WarehouseSOH,ItemName,Division,Department,Category,HouseBrandIndicator,RawMaterialIndicator,StockReorderIndicator,BuyerGroup,Local/Oversea,ProductEntity,RQStatus,ABC SKUs,Vendor Code,Vendor Name,Item Maintenance,Last RC Date,1st TR In Date,ADS Qty,RL91 Sales Qty,TR date New Item,ADS Qty New Item,Stock cover days,SOH_Condition,StockCoverDays_Condition,OOS_Status
str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,i64,i64,i64,f64,f64,bool,bool,str
"""72170896""","""WJI""",-20,"""ECO CNY TINSEL 200CM""","""GMS""","""HOME & LIVING""","""DECORATION""","""Y""","""N""","""N""","""YW""","""OVERSEA""","""Eco-Shop""","""Seasonal Item""","""Z""","""TPCHE0166""","""CHEN XIA LIAN""","""New Item + Eco-Shop""",null,null,64,2436,null,2436.0,-0.00821,true,true,"""OOS"""
"""40080023""","""WJI""",0,"""KREATI FANCY GIFT BOX""","""GMS""","""STATIONERY""","""GIFTING""","""Y""","""N""","""N""","""SH""","""OVERSEA""","""Eco-Shop""","""Warehouse Clearance""","""Z""","""TPGLO0008""","""GLORYTINS""","""New Item + Eco-Shop""",null,null,73,6610,null,6610.0,0.0,true,true,"""OOS"""
"""65050141""","""WKL""",48,"""CASA SEASONING BOX WHITE 8499 …","""GMS""","""KITCHENWARE""","""PLASTICWARE""","""Y""","""N""","""Y""","""YW""","""OVERSEA""","""Eco-Shop""","""Active""","""Z""","""TPZHU0068""","""ZHU XIAO""","""New Item + Eco-Shop""",null,null,123,11185,null,11185.0,0.004291,true,true,"""OOS"""
"""72170710""","""WJI""",-12,"""ECO CNY HANG DÉCOR 35CM""","""GMS""","""HOME & LIVING""","""DECORATION""","""Y""","""N""","""N""","""YW""","""OVERSEA""","""Eco-Shop""","""Seasonal Item""","""Z""","""TPYEZ0003""","""YE ZHONG LI""","""New Item + Eco-Shop""",null,null,14,531,null,531.0,-0.022599,true,true,"""OOS"""
"""72170640""","""WJI""",-190,"""ECO CNY LED LANTERN 8.5X10CM""","""GMS""","""HOME & LIVING""","""DECORATION""","""Y""","""N""","""N""","""YW""","""OVERSEA""","""Eco-Shop""","""Seasonal Item""","""Z""","""TPSHE0067""","""SHENG HAO GONG YI""","""New Item + Eco-Shop""",null,null,151,5593,null,5593.0,-0.033971,true,true,"""OOS"""
"""72170839""","""WJI""",-36,"""ECO CNY HANG DÉCOR 300CM""","""GMS""","""HOME & LIVING""","""DECORATION""","""Y""","""N""","""N""","""YW""","""OVERSEA""","""Eco-Shop""","""Seasonal Item""","""Z""","""TPMAQ0002""","""MA QIONG HUA""","""New Item + Eco-Shop""",null,null,19,714,null,714.0,-0.05042,true,true,"""OOS"""
"""72170546""","""WKL""",12,"""ECO CHRISTMAS STAR HANGING DEC…","""GMS""","""HOME & LIVING""","""DECORATION""","""Y""","""N""","""N""","""EPS-O""","""OVERSEA""","""Eco-Shop""","""Seasonal Item""","""Z""","""TPGON0012""","""GONG NAI MIN""","""New Item + Eco-Shop""",null,null,22,2033,null,2033.0,0.005903,true,true,"""OOS"""
"""72170877""","""WJI""",-12,"""ECO CNY HANG DÉCOR 50X30CM""","""GMS""","""HOME & LIVING""","""DECORATION""","""Y""","""N""","""N""","""YW""","""OVERSEA""","""Eco-Shop""","""Seasonal Item""","""Z""","""TPMAQ0002""","""MA QIONG HUA""","""New Item + Eco-Shop""",null,null,33,1220,null,1220.0,-0.009836,true,true,"""OOS"""


In [ ]:
## NEWEST BASED ON THE REQUIREMENTS

# Create new df which is AnalysisSOH_df from WH_soh. We will only take for RQStatus=Active, StockReorderIndicator=Y, Local/Oversea=LOCAL
AnalysisSOH_df = WH_soh.filter(
    (pl.col('RQStatus') == 'Active') &
    (pl.col('StockReorderIndicator') == 'Y') &
    (pl.col('Local/Oversea') == 'LOCAL') &
    (pl.col('OOS_Status') == 'OOS') &
    (pl.col('BuyerGroup').is_in(['EPS-A','L','EPS-L','EP-L','A'])))


In [ ]:
# Reorder columns arrangement for WH_soh
AnalysisSOH_df = AnalysisSOH_df.select([
    "Division",
    "ABC SKUs",
    "ItemNo",
    "ItemName",
    "Vendor Code",
    "Vendor Name",
    "WarehouseID",
    "Local/Oversea",
    "RQStatus",
    "ProductEntity",
    "BuyerGroup",
    "Department",
    "Category",
    "HouseBrandIndicator",
    "RawMaterialIndicator",
    "StockReorderIndicator",
    "Last RC Date",
    "1st TR In Date",
    "ADS Qty",
    "RL91 Sales Qty",
    "TR date New Item",
    "ADS Qty New Item",
    "WarehouseSOH",
    "Stock cover days",
    "SOH_Condition",
    "StockCoverDays_Condition",
    "OOS_Status",
    "Item Maintenance",

])

In [ ]:
# Check if the ItemNo got duplicated
AnalysisSOH_df.filter(AnalysisSOH_df['ItemNo'].is_duplicated())

Division,ABC SKUs,ItemNo,ItemName,Vendor Code,Vendor Name,WarehouseID,Local/Oversea,RQStatus,ProductEntity,BuyerGroup,Department,Category,HouseBrandIndicator,RawMaterialIndicator,StockReorderIndicator,Last RC Date,1st TR In Date,ADS Qty,RL91 Sales Qty,TR date New Item,ADS Qty New Item,WarehouseSOH,Stock cover days,SOH_Condition,StockCoverDays_Condition,OOS_Status,Item Maintenance
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,i64,i64,i64,f64,i64,f64,bool,bool,str,str


In [ ]:
# Try save as excel
AnalysisSOH_df.write_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\AnalysisSOH_df.xlsx")

## ORDERING REPORT SUMMARY

In [ ]:
# Load ordering summary to retrieve past ordering date
order_directory = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\OrderingWeekly\WeeklySummaryOrdering.xlsm"

# Select certain columns that were used for the analysis
select_cols = ["Week - EDD","Week","Division","ItemNo",'Item Name',
               "RQ Status","Delivery based on truck condition\n1. Full truck\n2. X Full truck\n3. Pallet",
               'WH ADJ qty (Last week)','Revised Rounded Order Quantity',
               'Order indicator','This Week Order Remark','Planner Remark',
               'Control store rate >2.0, total SOH ES >1k and EP >60PCS','Existing Reorder Y','Final Delivery Date']

order_df = pd.read_excel(order_directory,
                        sheet_name="Weekly Order Data",
                        skiprows=7,
                        usecols=select_cols,
                        dtype={'ItemNo':str},
                        engine='openpyxl')

order_df['WH ADJ qty (Last week)'] = pd.to_numeric(order_df['WH ADJ qty (Last week)'], errors='coerce').fillna(0).astype('int64')

In [ ]:
# List down all data to correct the reasoning, make it in correct spacing, spelling.
data = {
    'This Week Order Remark': [
        'All Order + DC SOH > Demand',
        'Does not meet Pallet MOQ requirement',
        'Does not meet Item MOQ requirement',
        'other',
        'Does not meet one carton requirement',
        'EP Store >2.0 and Total SOH >60 PCS',
        'Reorder No Item',
        'Does not meet Mix MOQ/MOV requirement',
        'Does not meet Truck MOQ requirement',
        'Relisting item',
        "Eco-Plus items with stores' stock turn days â‰¥ 60 days,  and company SOH > 60 units.",
        "Eco-Shop items with stores' stock turn days â‰¥ 60 days,  and company SOH > 1000 units.",
        'New Item, No Sales',
        'New Item, waiting buyer propose 1st PO',
        'Item D',
        'New Item',
        'No order due to high stock rate',
        'Does not meet item MOQ requirement',
        'Buyer request temporary onhold',
        'Eco-Plus items, stop order',
        'Clear balance, to be discontinue',
        'EP Stop order start from week 28',
        'new item',
        'New item; the expired purchase order has been canceled.',
        'Balance to clear, Item to be discontinue',
        'Does not  meet Mix MOQ/MOV requirement',
        'relisting item',
        'Does not meet Truck MOQ requiement',
        'All Order + DC SOH < Demand',
        'New Item, waiting buyer to propose 1st PO',
        'change Temporary on Hold',
        'New Item, waiting buyer to propose first PO',
        'New Item, the expired PO has been cancelled',
        'Buyer request to top up order quantity',
        'Supplier informed this item was phased out',
        'WH rate > 2.0 ref actual sales',
        'Balance was cleared',
        "New Item, Outlet haven't receive stock",
        'Store rate >2.0 ref actual sales',
        'Month Stock Rate > 2.0 ref actual sales',
        'Meet Item MOQ requirement',
        'Seasonal Items',
        'Back to School',
        'NEW ITEM',
        'pallet',
        'Stop ordering for all Eco-Plus items',
        'Back to School Items',
        'DC soh rate high, InStock After Arrival high',
        'Non-Inventory',
        'Pallet count + No order',
        'Seasonal Item',
        'Balance to clear, Item to be discontinued'
    ]
}



# Define a dictionary with specific replacements for known variations
replacement_map = {
    # Standardize common variations to consistent forms
    'New Item': ['new item', 'NEW ITEM', 'New Item', 'New Item, No Sales', 'new item'],
    'Does not meet Mix MOQ/MOV requirement': [
        'Does not meet Mix MOQ/MOV requirement', 'Does not  meet Mix MOQ/MOV requirement'
    ],
    'Does not meet MIX MOQ/MOV requirement': [
        'Does not meet Mix MOQ/MOV requirement', 'Does not  meet Mix MOQ/MOV requirement','Does not meet MIX MOQ requirement'
    ],
    'Does not meet Truck MOQ requirement': [
        'Does not meet Truck MOQ requirement', 'Does not meet Truck MOQ requiement'
    ],
    'Relisting item': ['Relisting item', 'relisting item'],
    'Does not meet Item MOQ requirement': [
        'Does not meet Item MOQ requirement', 'Does not meet item MOQ requirement'
    ],
    'Back to School': ['Back to School', 'Back to School Items'],
    "Eco-Plus items with stores' stock turn days â‰¥ 60 days,  and company SOH > 60 units.": [
        'EP Store >2.0 and Total SOH >60 PCS',"Eco-Plus items with stores' stock turn days â‰¥ 60 days,  and company SOH > 60 units.",'EP Store >2.0 and Total SOH >60 PCS'],
    'Balance to clear, Item to be discontinue': [
        'Balance to clear, Item to be discontinue', 'Clear balance, to be discontinue', 'Balance to clear, Item to be discontinued','Balance to clear, item to be discontinued','Balance to clear, item to be discontinue'
    ],
    'Meet Mix MOQ/MOV requirement':[
        'Meet Mix MOQ requirement','Meet MIX MOQ requirements',"Meet Mix MOQ/MOV requirement"
    ]
}

# Flatten the dictionary for replacement
final_replacement_map = {variant: standard for standard, variants in replacement_map.items() for variant in variants}

# Apply replacements to the "This Week Order Remark" column
order_df['This Week Order Remark'] = order_df['This Week Order Remark'].replace(final_replacement_map)

# Rename array(['Order', 'No Order', 'No ORDER', 'order', 'Done Order','Done order', 'Pending', 'ORDER'] to standard. Order & No Order & Done Order & Pending in Order indicator
order_df['Order indicator'] = order_df['Order indicator'].replace(['Order', 'No Order', 'No ORDER', 'order', 'Done Order','Done order', 'Pending', 'ORDER'], ['Order', 'No Order', 'No Order', 'Order', 'Done Order', 'Done Order', 'Pending', 'Order'])

In [ ]:
order_df['Week'].unique()

array([22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40,  1, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52],
      dtype=int64)

In [ ]:
# Create new dataframe of order_df with only Week are past 4 weeks
orderReport = order_df[order_df['Week'].isin([one_week_ago, two_week_ago, three_week_ago, four_week_ago])]

In [ ]:
# Combine and Check RQ Status Changes for past 4 week by ItemNo
orderReport['RQ Status'] = orderReport['RQ Status'].apply(lambda x: str(x) if isinstance(x, float) else x)
orderReport['Week-RQ Status'] = orderReport['Week'].astype(str) + '-' + orderReport['RQ Status']

# Group by 'ItemNo' and join the 'Week-RQ Status'
orderReport['RQ Status Changes'] = orderReport.groupby('ItemNo')['Week-RQ Status'].transform(lambda x: ', '.join(x))

# Drop duplicates if only unique 'ItemNo' entries are needed with their RQ status changes
orderReport[['ItemNo', 'RQ Status Changes']].drop_duplicates().reset_index(drop=True)

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_38320\1270870704.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_38320\1270870704.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_38320\1270870704.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,ItemNo,RQ Status Changes
0,10055012,"1-Supplier Issue, 50-Supplier Issue, 51-Suppli..."
1,11105118,"1-Active, 50-Active, 51-Active, 52-Active"
2,11105119,"1-Active, 50-Active, 51-Active, 52-Active"
3,11495002,"1-Active, 50-Active, 51-Active, 52-Active"
4,99405010,"1-Non-Inventory, 50-Non-Inventory, 51-nan, 52-..."
...,...,...
3933,65165297,50-Active
3934,41525005,50-Active
3935,64253032,"50-Active, 51-Active, 52-Active"
3936,64243047,51-Active


In [ ]:
orderReport['RQ Status Changes'].unique()

array(['1-Supplier Issue, 50-Supplier Issue, 51-Supplier Issue, 52-Supplier Issue',
       '1-Active, 50-Active, 51-Active, 52-Active',
       '1-Non-Inventory, 50-Non-Inventory, 51-nan, 52-Non-Inventory',
       '1-Active',
       '1-Supplier Issue, 50-Active, 51-Active, 52-Supplier Issue',
       '1-Non-Inventory, 52-Active', '1-Active, 52-Active',
       '1-Active, 51-Active, 52-Active',
       '1-Non-Inventory, 50-Non-Inventory, 51-nan, 52-Non-Inventory, 1-Non-Inventory, 50-Non-Inventory, 51-Non-Inventory, 52-Non-Inventory',
       '50-Active, 51-Active, 52-Temporary on Hold',
       '50-Active, 51-Active, 52-Active', '50-Active, 51-Active',
       '50-Active', '50-Non-Inventory, 51-nan, 52-Non-Inventory',
       '50-Non-Inventory, 51-nan, 52-Non-Inventory, 1-Non-Inventory, 50-Non-Inventory, 51-Non-Inventory, 52-Non-Inventory',
       '51-Active',
       '51-nan, 52-Non-Inventory, 1-Non-Inventory, 50-Non-Inventory, 51-Non-Inventory, 52-Non-Inventory',
       '52-Active, 1-Non-Inven

In [ ]:
# Add the Delivery Date Week column with the week number
orderReport['Delivery Date Week'] = (
    pd.to_datetime(orderReport['Final Delivery Date'], errors='coerce')  # Ensure datetime conversion
    .dt.strftime('%Y-%m-%d') +  # Convert back to string format
    " (WK" +
    pd.to_datetime(orderReport['Final Delivery Date'], errors='coerce').dt.isocalendar().week.astype(str) + ")"  # Add week number
)

In [ ]:
display(orderReport)

,Week - EDD,Week,Division,ItemNo,Item Name,RQ Status,Delivery based on truck condition\n1. Full truck\n2. X Full truck\n3. Pallet,WH ADJ qty (Last week),Revised Rounded Order Quantity,Final Delivery Date,Order indicator,This Week Order Remark,Planner Remark,"Control store rate >2.0, total SOH ES >1k and EP >60PCS",Existing Reorder Y,Delivery Date Week
76213,1-2,1,FOOD,10055012,A1 SNACK KUACI MARJERIN 90G,Supplier Issue,No Delivery Conditions,0,12600,2025-01-09,Order,All Order + DC SOH < Demand,NaN,NaN,Existing Reorder Y,2025-01-09 (WK2)
76214,1-2,1,FOOD,11105118,ABLE FARM VEGETABLE COOKING OIL 350ML,Active,Pallet,0,7200,2025-01-10,Order,Meet Pallet MOQ requirement,NaN,NaN,Existing Reorder Y,2025-01-10 (WK2)
76215,1-2,1,FOOD,11105119,ABLE FARM SUNFLOWER OIL 250ML,Active,Pallet,0,2400,2025-01-10,Order,All Order + DC SOH < Demand,NaN,NaN,Existing Reorder Y,2025-01-10 (WK2)
76216,1-2,1,FOOD,11495002,ECOSAFA 100% MINYAK MASAK SAYURAN 250ML,Active,Pallet,0,2400,2025-01-10,Order,All Order + DC SOH < Demand,NaN,NaN,Existing Reorder Y,2025-01-10 (WK2)
76217,1-2,1,NON TRADE,99405010,Pallet Others Able Perfect Sdn Bhd,Non-Inventory,NaN,0,0,2025-01-10,Order,pallet,PP=0,NaN,Existing Reorder Y,2025-01-10 (WK2)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130675,52-3,52,GMS,62035113,BICOLOUR CORN FRUITY SEED GRAB-SD-FR-006 18SEE...,Active,No Delivery Conditions,0,0,2025-01-16,No Order,All Order + DC SOH > Demand,NaN,NaN,Existing Reorder Y,2025-01-16 (WK3)
130676,52-3,52,GMS,62035118,FIVE ANGLE OKRA VEGE SEED GRAB-SD-FV-008 18SEE...,Active,No Delivery Conditions,0,0,2025-01-16,No Order,All Order + DC SOH > Demand,NaN,NaN,Existing Reorder Y,2025-01-16 (WK3)
130677,52-3,52,GMS,62035119,CABBAGE VEGE GRAB-SD-LV-022 250SEEDS+- 11CM X...,Active,No Delivery Conditions,0,0,2025-01-16,No Order,All Order + DC SOH > Demand,NaN,NaN,Existing Reorder Y,2025-01-16 (WK3)
130678,52-5,52,NON FOOD,34015002,PETANI MINYAK GAMAT PLUS 60ML,Active,No Delivery Conditions,0,0,2025-01-31,No Order,All Order + DC SOH > Demand,NaN,NaN,Existing Reorder Y,2025-01-31 (WK5)


In [ ]:
# Create new column which is "Week-Order Remark" which combine between Week and This Week Order Remark
orderReport['Week-Order Remark'] = orderReport['Week'].astype(str) + " - " + orderReport['This Week Order Remark'].astype(str)

# Create slow moving item based on column ontrol store rate >2.0, total SOH ES >1k and EP >60PCS which is Cannot Order
orderReport['Slow Moving Item'] = orderReport['Week'].astype(str) + " - " + orderReport['Control store rate >2.0, total SOH ES >1k and EP >60PCS'].astype(str)

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\1914948744.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\1914948744.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Define the list of MOQ conditions to search for
MOQ_conditions = [
    'Does not meet Item MOQ requirement',
    'Does not meet Pallet MOQ requirement',
    'Does not meet Mix MOQ/MOV requirement',
    'Does not meet Truck MOQ requirement',
    'Does not meet one carton requirement',
]

# Create a pattern to match any of the MOQ conditions
pattern = '|'.join(MOQ_conditions)

# Debugging: Print the DataFrame columns
print(orderReport.columns)

# Function to generate the "MOQ Delivery Conditions" string
def create_moq_string(row):
    if pd.isna(row['Week-Order Remark']) or not isinstance(row['Week-Order Remark'], str):
        return f"No- Not MOQ Delivery in {row['Week']}"

    # Check if any MOQ condition matches
    if any(cond in row['Week-Order Remark'] for cond in MOQ_conditions):
        # Extract weeks from Week-Order Remark
        weeks = [wk.split('-')[0].strip() for wk in row['Week-Order Remark'].split(',')]
        week_str = ', '.join([f"Week {week}" for week in weeks])

        # Append the delivery date and its corresponding week
        return (f"Yes- Not meet MOQ Delivery in {week_str} - should be delivered in "
                f"{row['Delivery Date Week']}")
    else:
        return f"No- Not MOQ Delivery in {row['Week']}"

# Ensure 'MOQ Delivery Conditions' column is created properly
orderReport['MOQ Delivery Conditions'] = orderReport.apply(create_moq_string, axis=1)

orderReport.columns = orderReport.columns.str.strip()
orderReport['MOQ Delivery Conditions'] = orderReport['MOQ Delivery Conditions'].astype(str)

# Verify column creation
if 'MOQ Delivery Conditions' not in orderReport.columns:
    raise ValueError("'MOQ Delivery Conditions' column not created successfully.")

def summarize_moq_conditions(group):
    print(f"Processing group for ItemNo: {group.name}")
    print(group)
    relevant_conditions = group.str.contains(r'Yes- Not meet MOQ Delivery', na=False)
    filtered_conditions = group[relevant_conditions]

    if not filtered_conditions.empty:
        return ' & '.join(filtered_conditions.unique())
    else:
        return "No- Not MOQ Delivery"

# Apply the grouping and create the new column
orderReport['MOQ Delivery Indicators'] = (
    orderReport.groupby('ItemNo')['MOQ Delivery Conditions']
    .transform(summarize_moq_conditions)
)

# Verify the output
print(orderReport[['ItemNo', 'Week-Order Remark', 'MOQ Delivery Conditions', 'MOQ Delivery Indicators']].head())

Index(['Week - EDD', 'Week', 'Division', 'ItemNo', 'Item Name', 'RQ Status',
       'Delivery based on truck condition\n1. Full truck\n2. X Full truck\n3. Pallet',
       'WH ADJ qty (Last week)', 'Revised Rounded Order Quantity',
       'Final Delivery Date', 'Order indicator', 'This Week Order Remark',
       'Planner Remark',
       'Control store rate >2.0, total SOH ES >1k and EP >60PCS',
       'Existing Reorder Y', 'Delivery Date Week', 'Week-Order Remark',
       'Slow Moving Item'],
      dtype='object')


C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\3126579881.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\3126579881.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Streaming output truncated to the last 5000 lines.
127765    No- Not MOQ Delivery in 51
129908    No- Not MOQ Delivery in 52
Name: 64135005, dtype: object
Processing group for ItemNo: 64135007
103798                            No- Not MOQ Delivery in 1
125193    Yes- Not meet MOQ Delivery in Week 50 - should...
127333                           No- Not MOQ Delivery in 51
129489                           No- Not MOQ Delivery in 52
Name: 64135007, dtype: object
Processing group for ItemNo: 64135008
103799     No- Not MOQ Delivery in 1
125194    No- Not MOQ Delivery in 50
127332    No- Not MOQ Delivery in 51
129490    No- Not MOQ Delivery in 52
Name: 64135008, dtype: object
Processing group for ItemNo: 64135009
103800     No- Not MOQ Delivery in 1
125195    No- Not MOQ Delivery in 50
127338    No- Not MOQ Delivery in 51
129491    No- Not MOQ Delivery in 52
Name: 64135009, dtype: object
Processing group for ItemNo: 64135015
104533     No- Not MOQ Delivery in 1
125923    No- Not MOQ Delivery

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\3126579881.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
orderReport['MOQ Delivery Indicators'].unique()

array(['No- Not MOQ Delivery',
       'Yes- Not meet MOQ Delivery in Week 52 - should be delivered in 2025-01-10 (WK2)',
       'Yes- Not meet MOQ Delivery in Week 50 - should be delivered in 2025-01-03 (WK1)',
       'Yes- Not meet MOQ Delivery in Week 52 - should be delivered in 2025-01-17 (WK3)',
       'Yes- Not meet MOQ Delivery in Week 51 - should be delivered in 2025-01-20 (WK4) & Yes- Not meet MOQ Delivery in Week 52 - should be delivered in 2025-01-27 (WK5)',
       'Yes- Not meet MOQ Delivery in Week 50 - should be delivered in 2024-12-31 (WK1)',
       'Yes- Not meet MOQ Delivery in Week 52 - should be delivered in 2025-02-26 (WK9)',
       'Yes- Not meet MOQ Delivery in Week 52 - should be delivered in 2024-12-31 (WK1)',
       'Yes- Not meet MOQ Delivery in Week 1 - should be delivered in 2025-01-23 (WK4)',
       'Yes- Not meet MOQ Delivery in Week 52 - should be delivered in 2025-01-04 (WK1)',
       'Yes- Not meet MOQ Delivery in Week 1 - should be delivered in 2025-01-

In [ ]:
# save orderReport to check on wheter MOQ delivery condition correct or not
orderReport.to_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\order_df.xlsx")

In [ ]:
# Extract Relisting Weeks directly into a new column
orderReport['Relisting Weeks'] = orderReport['Week-Order Remark'].str.extract(r"(?i).*?(\d+).*Relisting Item", expand=False)

# Group by ItemNo and summarize weeks
def summarize_relisting_weeks(group):
    valid_weeks = group['Relisting Weeks'].dropna().unique()  # Get unique, non-null weeks
    if len(valid_weeks) > 0:
        sorted_weeks = sorted(valid_weeks, key=int)  # Sort numerically
        return f"Yes- Relisting Item in weeks {', '.join(sorted_weeks)}"
    return "No- Not Relisting Item"

# Apply the summarization function grouped by 'ItemNo'
orderReport['Relisting Item Indicator'] = (
    orderReport.groupby('ItemNo')['Relisting Weeks']
    .transform(lambda group: summarize_relisting_weeks(orderReport[orderReport['ItemNo'] == group.name]))
)

# Drop intermediate column if not needed
orderReport = orderReport.drop(columns=['Relisting Weeks'])

# Final output
print(orderReport[['ItemNo', 'Relisting Item Indicator']])

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\2733561146.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



          ItemNo Relisting Item Indicator
76213   10055012   No- Not Relisting Item
76214   11105118   No- Not Relisting Item
76215   11105119   No- Not Relisting Item
76216   11495002   No- Not Relisting Item
76217   99405010   No- Not Relisting Item
...          ...                      ...
130675  62035113   No- Not Relisting Item
130676  62035118   No- Not Relisting Item
130677  62035119   No- Not Relisting Item
130678  34015002   No- Not Relisting Item
130679  34015003   No- Not Relisting Item

[15436 rows x 2 columns]


C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\2733561146.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
orderReport['Relisting Item Indicator'].unique()

array(['No- Not Relisting Item', 'Yes- Relisting Item in weeks 1',
       'Yes- Relisting Item in weeks 52',
       'Yes- Relisting Item in weeks 51',
       'Yes- Relisting Item in weeks 50',
       'Yes- Relisting Item in weeks 50, 51',
       'Yes- Relisting Item in weeks 51, 52',
       'Yes- Relisting Item in weeks 1, 52',
       'Yes- Relisting Item in weeks 1, 50, 52',
       'Yes- Relisting Item in weeks 1, 50, 51, 52',
       'Yes- Relisting Item in weeks 1, 51',
       'Yes- Relisting Item in weeks 50, 51, 52'], dtype=object)

In [ ]:
# 1. Modify "Period Slow Moving" code to include the default "Not slow moving item" message
orderReport['Period Slow Moving'] = orderReport.groupby('ItemNo')['Slow Moving Item'].transform(
    lambda x: "Slow moving item on Weeks " + ", ".join(
        sorted(set(wk.split(' - ')[0] for wk in x.dropna() if 'Cannot Order' in wk))
    ) if any('Cannot Order' in str(item) for item in x.dropna()) else "Not Slow Moving Item"
)

# Convert 'Period Slow Moving' to string type before applying .str.contains
orderReport['Period Slow Moving'] = orderReport['Period Slow Moving'].astype(str)

# 2. After It got determined like Slow moving item on weeks ...., create new column which is 'Slow Moving Item Indicator', categorize it as Yes-Slow Moving Item else No-Not Slow Moving Item
orderReport['Slow Moving Item Indicator'] = np.where(
    orderReport['Period Slow Moving'].str.contains('Slow moving item on Weeks', na=False),
    'Yes-Slow Moving Item',
    'No-Not Slow Moving Item'
)

In [ ]:
# Step 2: Add 'Balance To Clear' column with weeks where the remark is present
orderReport['Balance To Clear'] = orderReport.groupby('ItemNo')['Week-Order Remark'].transform(
    lambda remarks: "Balance to clear in weeks " + ", ".join(
        sorted(set(wk.split(' - ')[0] for wk in remarks if isinstance(wk, str) and 'Balance to clear' in wk))
    ) if any('Balance to clear' in str(wk) for wk in remarks) else ''
)

# Step 2: Add 'Item To Clear Indicator' column based on the 'Balance To Clear' column
orderReport['Item To Clear Indicator'] = np.where(
    orderReport['Balance To Clear'] != '',  # Check if 'Balance To Clear' is non-empty
    'Yes- Balance To Clear Item',
    'No- Not Balance To Clear Item'
)

In [ ]:
display(orderReport)

,Week - EDD,Week,Division,ItemNo,Item Name,RQ Status,Delivery based on truck condition\n1. Full truck\n2. X Full truck\n3. Pallet,WH ADJ qty (Last week),Revised Rounded Order Quantity,Final Delivery Date,...,Delivery Date Week,Week-Order Remark,Slow Moving Item,MOQ Delivery Conditions,MOQ Delivery Indicators,Relisting Item Indicator,Period Slow Moving,Slow Moving Item Indicator,Balance To Clear,Item To Clear Indicator
76213,1-2,1,FOOD,10055012,A1 SNACK KUACI MARJERIN 90G,Supplier Issue,No Delivery Conditions,0,12600,2025-01-09,...,2025-01-09 (WK2),1 - All Order + DC SOH < Demand,1 - nan,No- Not MOQ Delivery in 1,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
76214,1-2,1,FOOD,11105118,ABLE FARM VEGETABLE COOKING OIL 350ML,Active,Pallet,0,7200,2025-01-10,...,2025-01-10 (WK2),1 - Meet Pallet MOQ requirement,1 - nan,No- Not MOQ Delivery in 1,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
76215,1-2,1,FOOD,11105119,ABLE FARM SUNFLOWER OIL 250ML,Active,Pallet,0,2400,2025-01-10,...,2025-01-10 (WK2),1 - All Order + DC SOH < Demand,1 - nan,No- Not MOQ Delivery in 1,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
76216,1-2,1,FOOD,11495002,ECOSAFA 100% MINYAK MASAK SAYURAN 250ML,Active,Pallet,0,2400,2025-01-10,...,2025-01-10 (WK2),1 - All Order + DC SOH < Demand,1 - nan,No- Not MOQ Delivery in 1,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
76217,1-2,1,NON TRADE,99405010,Pallet Others Able Perfect Sdn Bhd,Non-Inventory,NaN,0,0,2025-01-10,...,2025-01-10 (WK2),1 - pallet,1 - nan,No- Not MOQ Delivery in 1,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130675,52-3,52,GMS,62035113,BICOLOUR CORN FRUITY SEED GRAB-SD-FR-006 18SEE...,Active,No Delivery Conditions,0,0,2025-01-16,...,2025-01-16 (WK3),52 - All Order + DC SOH > Demand,52 - nan,No- Not MOQ Delivery in 52,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
130676,52-3,52,GMS,62035118,FIVE ANGLE OKRA VEGE SEED GRAB-SD-FV-008 18SEE...,Active,No Delivery Conditions,0,0,2025-01-16,...,2025-01-16 (WK3),52 - All Order + DC SOH > Demand,52 - nan,No- Not MOQ Delivery in 52,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
130677,52-3,52,GMS,62035119,CABBAGE VEGE GRAB-SD-LV-022 250SEEDS+- 11CM X...,Active,No Delivery Conditions,0,0,2025-01-16,...,2025-01-16 (WK3),52 - All Order + DC SOH > Demand,52 - nan,No- Not MOQ Delivery in 52,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item
130678,52-5,52,NON FOOD,34015002,PETANI MINYAK GAMAT PLUS 60ML,Active,No Delivery Conditions,0,0,2025-01-31,...,2025-01-31 (WK5),52 - All Order + DC SOH > Demand,52 - nan,No- Not MOQ Delivery in 52,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,,No- Not Balance To Clear Item


In [ ]:
orderReport[orderReport['This Week Order Remark']=='Balance to clear, Item to be discontinue'].sample(2)

,Week - EDD,Week,Division,ItemNo,Item Name,RQ Status,Delivery based on truck condition\n1. Full truck\n2. X Full truck\n3. Pallet,WH ADJ qty (Last week),Revised Rounded Order Quantity,Final Delivery Date,...,Delivery Date Week,Week-Order Remark,Slow Moving Item,MOQ Delivery Conditions,MOQ Delivery Indicators,Relisting Item Indicator,Period Slow Moving,Slow Moving Item Indicator,Balance To Clear,Item To Clear Indicator
124804,50-52,50,GMS,65015248,CASA ROUND CONTAINER CRD250 8PCS 12x12CM,Active,No Delivery Conditions,0,10000,2024-12-28,...,2024-12-28 (WK52),"50 - Balance to clear, Item to be discontinue",50 - nan,No- Not MOQ Delivery in 50,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,Balance to clear in weeks 50,Yes- Balance To Clear Item
126846,51-2,51,GMS,64545003,ECO HEXAGON CHARCOAL BRIQUETTE 1KG 12PCS,Warehouse Clearance,Full Truck,0,4608,2025-01-09,...,2025-01-09 (WK2),"51 - Balance to clear, Item to be discontinue",51 - nan,No- Not MOQ Delivery in 51,No- Not MOQ Delivery,No- Not Relisting Item,Not Slow Moving Item,No-Not Slow Moving Item,Balance to clear in weeks 51,Yes- Balance To Clear Item


In [ ]:
# Convert order_df to polars before merge with minmax
orderReport = pl.from_pandas(orderReport)

In [ ]:
# Merge minmax triggered in order_df
orderReport = orderReport.join(minmax4weeks.with_columns(minmax4weeks['Week'].cast(pl.Int64)),
                         on=['ItemNo','Week'], how='left')

# Keep week1triggersoft to be integer
orderReport = orderReport.with_columns(
    pl.col("week1triggersoft").fill_null(0).cast(pl.Int64) #Use Polars fill_null and cast methods
)

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\3652702590.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Only retrieve when got Relistem Item inside Week-Order Remark. This one case insensitive. Retrieve all
relisting_summary = orderReport.filter(pl.col('Week-Order Remark').str.to_lowercase().str.contains('relisting item'))

RelistSummary = (
    relisting_summary
    .group_by("ItemNo")
    .agg([
        # Concatenate "Week" and "" as [Week-Order Remark] for each entry
        pl.concat_str(["Week", pl.lit("-"), "This Week Order Remark"], separator="")
          .implode()
          .alias("Relisting Item Listing")
    ])
)

orderReport = orderReport.join(RelistSummary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\471373803.py:15: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Step 1: Create the grouped summary
whAdjSummary = (
    orderReport
    .group_by("ItemNo")
    .agg([
        pl.concat_str(["Week", pl.lit("-"), "WH ADJ qty (Last week)"], separator="")
          .implode()
          .alias("WH ADJ qty Summary")
    ])
)

# Step 2: Left join the summary back to the original DataFrame to add the new column
orderReport = orderReport.join(whAdjSummary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\2841941685.py:13: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
triggerSummary = (
    orderReport
    .group_by("ItemNo")
    .agg([
        # Concatenate "Week" and "This Week Order Remark" as [Week-Order Remark] for each entry
        pl.concat_str(["Week", pl.lit("-"), "week1triggersoft"], separator="")
          .implode()
          .alias("Triggered Qty Summary")
    ])
)

orderReport = orderReport.join(triggerSummary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\1584435160.py:12: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Step 1: Combine "Week" and "WH ADJ qty (Last week)" into a new column
orderReport = orderReport.with_columns(
    (pl.col("Week").cast(pl.Utf8) + " - " + pl.col("WH ADJ qty (Last week)").fill_null("No Adjustment").cast(pl.Utf8))
    .alias("Week-WH ADJ qty (Last week)")
)

# Step 2: Calculate the percentage of WH ADJ qty (Last week) relative to week1triggersoft
orderReport = orderReport.with_columns(
    pl.when((pl.col("WH ADJ qty (Last week)").is_not_null()) &
            (pl.col("week1triggersoft").is_not_null()) &
            (pl.col("week1triggersoft") != 0))
    .then((pl.col("WH ADJ qty (Last week)") / pl.col("week1triggersoft") * 100).round(2).cast(pl.Utf8) + "%")
    .otherwise(pl.lit("No Data"))
    .alias("WH Adjustment (Write Off)")
)

# Step 3: Create the WH Adjustment Category column with literal values using pl.lit()
orderReport = orderReport.with_columns(
    pl.when((pl.col("WH ADJ qty (Last week)") == 0) & (pl.col("week1triggersoft") == 0))
    .then(pl.lit("<30% Adjustment not affect OOS"))
    .when((pl.col("WH ADJ qty (Last week)").is_not_null()) &
          (pl.col("WH ADJ qty (Last week)") >= (pl.col("week1triggersoft") * 0.3)))
    .then(pl.lit(">30% Adjustment Affect OOS"))
    .when((pl.col("WH ADJ qty (Last week)").is_not_null()) &
          (pl.col("WH ADJ qty (Last week)") < (pl.col("week1triggersoft") * 0.3)))
    .then(pl.lit("<30% Adjustment not affect OOS"))
    .otherwise(pl.lit("No Adjustment"))
    .alias("WH Adjustment Category")
)

In [ ]:
# Instead of using 'replace', use 'with_replacement' in Polars
orderReport['MOQ Delivery Conditions'].unique()

MOQ Delivery Conditions
str
"""Yes- Not meet MOQ Delivery in …"
"""No- Not MOQ Delivery in 52"""
"""Yes- Not meet MOQ Delivery in …"
"""No- Not MOQ Delivery in 1"""
"""Yes- Not meet MOQ Delivery in …"
…
"""No- Not MOQ Delivery in 51"""
"""Yes- Not meet MOQ Delivery in …"
"""Yes- Not meet MOQ Delivery in …"


In [ ]:
AdjSummary = (
    orderReport
    .group_by("ItemNo")
    .agg([
        # Concatenate "Week" and "WH Adjustment Category" as [Week-Adj] for each entry
        pl.concat_str(["Week", pl.lit("-"), "WH Adjustment Category"], separator="")
          .implode()
          .alias("Adjustment Weekly Summary")
    ])
)

orderReport = orderReport.join(AdjSummary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\4035360437.py:12: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
orderReport.filter(pl.col('ItemNo')=='14085055').sample(3)

Week - EDD,Week,Division,ItemNo,Item Name,RQ Status,Delivery based on truck condition 1. Full truck 2. X Full truck 3. Pallet,WH ADJ qty (Last week),Revised Rounded Order Quantity,Final Delivery Date,Order indicator,This Week Order Remark,Planner Remark,"Control store rate >2.0, total SOH ES >1k and EP >60PCS",Existing Reorder Y,Delivery Date Week,Week-Order Remark,Slow Moving Item,MOQ Delivery Conditions,MOQ Delivery Indicators,Relisting Item Indicator,Period Slow Moving,Slow Moving Item Indicator,Balance To Clear,Item To Clear Indicator,week1triggersoft,Relisting Item Listing,WH ADJ qty Summary,Triggered Qty Summary,Week-WH ADJ qty (Last week),WH Adjustment (Write Off),WH Adjustment Category,Adjustment Weekly Summary
str,i64,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,list[list[str]],list[list[str]],list[list[str]],str,str,str,list[list[str]]
"""51-12""",51,"""FOOD""","""14085055""","""DELICIES SWEETENED CREAMER 390…","""Active""","""Item MOQ""",0,0,"""2025-03-21""","""No Order""","""All Order + DC SOH > Demand""",null,null,"""Existing Reorder Y""","""2025-03-21 (WK12)""","""51 - All Order + DC SOH > Dema…","""51 - nan""","""No- Not MOQ Delivery in 51""","""No- Not MOQ Delivery""","""No- Not Relisting Item""","""Not Slow Moving Item""","""No-Not Slow Moving Item""","""""","""No- Not Balance To Clear Item""",20688,null,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-16416"", ""50-13536"", … ""52-23520""]]","""51 - 0""","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]"
"""52-13""",52,"""FOOD""","""14085055""","""DELICIES SWEETENED CREAMER 390…","""Active""","""Item MOQ""",0,145152,"""2025-03-28""","""Order""","""Meet Item MOQ requirement""",null,null,"""Existing Reorder Y""","""2025-03-28 (WK13)""","""52 - Meet Item MOQ requirement""","""52 - nan""","""No- Not MOQ Delivery in 52""","""No- Not MOQ Delivery""","""No- Not Relisting Item""","""Not Slow Moving Item""","""No-Not Slow Moving Item""","""""","""No- Not Balance To Clear Item""",23520,null,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-16416"", ""50-13536"", … ""52-23520""]]","""52 - 0""","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]"
"""1-14""",1,"""FOOD""","""14085055""","""DELICIES SWEETENED CREAMER 390…","""Active""","""Item MOQ""",0,0,"""2025-04-04""","""No Order""","""All Order + DC SOH > Demand""",null,null,"""Existing Reorder Y""","""2025-04-04 (WK14)""","""1 - All Order + DC SOH > Deman…","""1 - nan""","""No- Not MOQ Delivery in 1""","""No- Not MOQ Delivery""","""No- Not Relisting Item""","""Not Slow Moving Item""","""No-Not Slow Moving Item""","""""","""No- Not Balance To Clear Item""",16416,null,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-16416"", ""50-13536"", … ""52-23520""]]","""1 - 0""","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]"


In [ ]:
orderReport = orderReport.select([
    "Week - EDD",
    "Week",
    "Division",
    "ItemNo",
    "Item Name",
    "RQ Status",
    "Delivery based on truck condition\n1. Full truck\n2. X Full truck\n3. Pallet",
    "Revised Rounded Order Quantity",
    "Order indicator",
    "This Week Order Remark",
    "Planner Remark",
    "Week-Order Remark",
    "Delivery Date Week",
    "Existing Reorder Y",
    'Relisting Item Indicator',
    'Relisting Item Listing',
    "Control store rate >2.0, total SOH ES >1k and EP >60PCS",
    "Slow Moving Item",
    "Slow Moving Item Indicator",
    "Period Slow Moving",
    "Item To Clear Indicator",
    "Balance To Clear",
    "MOQ Delivery Conditions",
    'MOQ Delivery Indicators',
    "WH ADJ qty (Last week)",
    "week1triggersoft",
    'WH ADJ qty Summary',
    'Triggered Qty Summary',
    "WH Adjustment (Write Off)",
    "WH Adjustment Category",
    "Adjustment Weekly Summary"

])

In [ ]:
# Try save the excel
orderReport.write_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\order_df1.xlsx")

In [ ]:
summaryOrder_df = orderReport.select([
    "ItemNo",
    'Relisting Item Indicator',
    'Relisting Item Listing',
    "Slow Moving Item",
    "Slow Moving Item Indicator",
    "Period Slow Moving",
    "Item To Clear Indicator",
    "Balance To Clear",
    "MOQ Delivery Conditions",
    'MOQ Delivery Indicators',
    'WH ADJ qty Summary',
    'Triggered Qty Summary',
    "WH Adjustment (Write Off)",
    "WH Adjustment Category",
    "Adjustment Weekly Summary"
])

In [ ]:
# Drop ItemNo or duplicated cause weeks summary, believe after those analysis and summary. Wont no need to return all rows. Just leave unique or once
summaryOrder_df = summaryOrder_df.unique(subset="ItemNo")

In [ ]:
# First, combine summaryOrder_df will merge with AnalysisSOH_df
AnalysisSOH_df = AnalysisSOH_df.join(summaryOrder_df, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\2601436813.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Save to excel
AnalysisSOH_df.write_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\After_AnalysisSOH_df.xlsx")

# ORDERING INDICATOR

In [ ]:
# Rearrange columns using .reindex
ordering_df = orderReport.select([
    "Week - EDD",
    "Week",
    "Division",
    "ItemNo",
    "Item Name",
    "RQ Status",
    "Delivery based on truck condition\n1. Full truck\n2. X Full truck\n3. Pallet",
    "Revised Rounded Order Quantity",
    "Order indicator",
    "This Week Order Remark",
    "Planner Remark",
    "Week-Order Remark",
    "Existing Reorder Y",
    'Relisting Item Indicator',
    'Relisting Item Listing',
    "Control store rate >2.0, total SOH ES >1k and EP >60PCS",
    "Slow Moving Item",
    "Slow Moving Item Indicator",
    "Period Slow Moving",
    "Item To Clear Indicator",
    "Balance To Clear",
    "MOQ Delivery Conditions",
    'MOQ Delivery Indicators',
    "WH ADJ qty (Last week)",
    "week1triggersoft",
    'WH ADJ qty Summary',
    'Triggered Qty Summary',
    "WH Adjustment (Write Off)",
    "WH Adjustment Category",
    "Adjustment Weekly Summary"

])

In [ ]:
# select_cols = ordering_df.select([
#     "ItemNo",
#     'Relisting Item Indicator',
#     'Relisting Item Listing',
#     "Slow Moving Item",
#     "Slow Moving Item Indicator",
#     "Period Slow Moving",
#     "Item To Clear Indicator",
#     "Balance To Clear",
#     "MOQ Delivery Conditions",
#     'MOQ Delivery Indicators',
#     'WH ADJ qty Summary',
#     'Triggered Qty Summary',
#     "WH Adjustment (Write Off)",
#     "WH Adjustment Category",
#     "Adjustment Weekly Summary"
# ])

In [ ]:
AnalysisSOH_df.sample(1)

Division,ABC SKUs,ItemNo,ItemName,Vendor Code,Vendor Name,WarehouseID,Local/Oversea,RQStatus,ProductEntity,BuyerGroup,Department,Category,HouseBrandIndicator,RawMaterialIndicator,StockReorderIndicator,Last RC Date,1st TR In Date,ADS Qty,RL91 Sales Qty,TR date New Item,ADS Qty New Item,WarehouseSOH,Stock cover days,SOH_Condition,StockCoverDays_Condition,OOS_Status,Item Maintenance,Relisting Item Indicator,Relisting Item Listing,Slow Moving Item,Slow Moving Item Indicator,Period Slow Moving,Item To Clear Indicator,Balance To Clear,MOQ Delivery Conditions,MOQ Delivery Indicators,WH ADJ qty Summary,Triggered Qty Summary,WH Adjustment (Write Off),WH Adjustment Category,Adjustment Weekly Summary
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,i64,i64,i64,f64,i64,f64,bool,bool,str,str,str,list[list[str]],str,str,str,str,str,str,str,list[list[str]],list[list[str]],str,str,list[list[str]]
"""FOOD""","""C""","""14019567""","""SO TINGE! C.WATERMELON LCHEE F…","""TPYEE0001""","""Yee Lee Trading Co. Sdn Bhd - …","""WJI""","""LOCAL""","""Active""","""Eco-Shop""","""EPS-L""","""GROCERY""","""READY TO DRINK (RTD)""","""N""","""N""","""Y""",2025-01-01,2021-05-23,58,5243,null,58.0,0,0.0,true,true,"""OOS""","""""","""No- Not Relisting Item""",null,"""1 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 1""","""No- Not MOQ Delivery""","[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-3720"", ""50-5496"", … ""52-6936""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]"


## NO-ORDER INDICATOR

In [ ]:
# Cases where the no order condition and easier to pull data
NoOrder_df = ordering_df.filter(ordering_df['Order indicator']=="No Order")

In [ ]:
NoOrder_df.sample(2)

Week - EDD,Week,Division,ItemNo,Item Name,RQ Status,Delivery based on truck condition 1. Full truck 2. X Full truck 3. Pallet,Revised Rounded Order Quantity,Order indicator,This Week Order Remark,Planner Remark,Week-Order Remark,Existing Reorder Y,Relisting Item Indicator,Relisting Item Listing,"Control store rate >2.0, total SOH ES >1k and EP >60PCS",Slow Moving Item,Slow Moving Item Indicator,Period Slow Moving,Item To Clear Indicator,Balance To Clear,MOQ Delivery Conditions,MOQ Delivery Indicators,WH ADJ qty (Last week),week1triggersoft,WH ADJ qty Summary,Triggered Qty Summary,WH Adjustment (Write Off),WH Adjustment Category,Adjustment Weekly Summary
str,i64,str,str,str,str,str,i64,str,str,str,str,str,str,list[list[str]],str,str,str,str,str,str,str,str,i64,i64,list[list[str]],list[list[str]],str,str,list[list[str]]
"""52-3""",52,"""FOOD""","""10075003""","""NESTLE KIT KAT 4 FINGER 35G""","""Active""","""No Delivery Conditions""",0,"""No Order""","""All Order + DC SOH > Demand""",null,"""52 - All Order + DC SOH > Dema…","""Existing Reorder Y""","""No- Not Relisting Item""",null,null,"""52 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 52""","""No- Not MOQ Delivery""",0,6696,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-6552"", ""50-5832"", … ""52-6696""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]"
"""52-3""",52,"""FOOD""","""80145084""","""TOYâ€™S CASTLE LIGHTNING WHEEL…","""Active""","""No Delivery Conditions""",0,"""No Order""","""All Order + DC SOH > Demand""",null,"""52 - All Order + DC SOH > Dema…","""Existing Reorder Y""","""No- Not Relisting Item""",null,null,"""52 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 52""","""No- Not MOQ Delivery""",0,864,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-360"", ""50-288"", … ""52-864""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]"


In [ ]:
# No order indicator for NoOrder_df
NoOrder_df = NoOrder_df.with_columns(
    pl.col("Week").cast(pl.Utf8)
)

# Create No Order Indicator for NoOrder_df
NoOrderSummary = (
    NoOrder_df.group_by('ItemNo')
    .agg(
        pl.col("Week")
        .unique()
        .sort()
        .alias("Weeks_No_Order")
    )
    .with_columns(
        pl.col("Weeks_No_Order")
        .map_elements(lambda x: f"No order on weeks {', '.join(x)}", return_dtype=pl.Utf8)
        .alias("No Order Indicator")
    )
)

NoOrder_df = NoOrder_df.join(NoOrderSummary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\720855337.py:22: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
display(NoOrder_df.sample(3))

Week - EDD,Week,Division,ItemNo,Item Name,RQ Status,Delivery based on truck condition 1. Full truck 2. X Full truck 3. Pallet,Revised Rounded Order Quantity,Order indicator,This Week Order Remark,Planner Remark,Week-Order Remark,Existing Reorder Y,Relisting Item Indicator,Relisting Item Listing,"Control store rate >2.0, total SOH ES >1k and EP >60PCS",Slow Moving Item,Slow Moving Item Indicator,Period Slow Moving,Item To Clear Indicator,Balance To Clear,MOQ Delivery Conditions,MOQ Delivery Indicators,WH ADJ qty (Last week),week1triggersoft,WH ADJ qty Summary,Triggered Qty Summary,WH Adjustment (Write Off),WH Adjustment Category,Adjustment Weekly Summary,Weeks_No_Order,No Order Indicator
str,str,str,str,str,str,str,i64,str,str,str,str,str,str,list[list[str]],str,str,str,str,str,str,str,str,i64,i64,list[list[str]],list[list[str]],str,str,list[list[str]],list[str],str
"""50-52""","""50""","""FOOD""","""10095073""","""627F1 MOROLL CRUNCH WAFER CHOC…","""Active""","""Pallet""",0,"""No Order""","""All Order + DC SOH > Demand""",null,"""50 - All Order + DC SOH > Dema…","""Existing Reorder Y""","""No- Not Relisting Item""",null,null,"""50 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 50""","""No- Not MOQ Delivery""",0,4176,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-4212"", ""50-4176"", … ""52-4104""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]","[""50"", ""52""]","""No order on weeks 50, 52"""
"""52-2""","""52""","""FOOD""","""14079543""","""DRINHO 300ML SOURSOP DRINK""","""Active""","""Full Truck""",0,"""No Order""","""All Order + DC SOH > Demand""",null,"""52 - All Order + DC SOH > Dema…","""Existing Reorder Y""","""No- Not Relisting Item""",null,null,"""52 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 52""","""No- Not MOQ Delivery""",0,3288,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-3768"", ""50-2904"", … ""52-3288""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]","[""50"", ""52""]","""No order on weeks 50, 52"""
"""51-52""","""51""","""FOOD""","""10095905""","""MUNCHYS OAT KRUNCH NUTTY CHOCO…","""Active""","""No Delivery Conditions""",0,"""No Order""","""All Order + DC SOH > Demand""",null,"""51 - All Order + DC SOH > Dema…","""Existing Reorder Y""","""No- Not Relisting Item""",null,null,"""51 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 51""","""No- Not MOQ Delivery""",0,6864,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-4488"", ""50-5448"", … ""52-6144""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]","[""1"", ""51"", ""52""]","""No order on weeks 1, 51, 52"""


In [ ]:
NoOrder_remarks = (
    NoOrder_df
    .group_by("ItemNo")
    .agg([
        # Concatenate "Week" and "This Week Order Remark" as [Week-Order Remark] for each entry
        pl.concat_str(["Week", pl.lit("-"), "This Week Order Remark"], separator="")
          .alias("Week-No Order Remark")
    ])
)

NoOrder_df = NoOrder_df.join(NoOrder_remarks, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\776387414.py:11: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
NoOrder_df['Week-No Order Remark'].explode().unique()

Week-No Order Remark
str
"""52-Reorder No Item"""
"""52-Eco-Plus items with stores'…"
"""1-pallet"""
"""50-New Item, waiting buyer pro…"
"""1-All Order + DC SOH > Demand"""
…
"""1-Relisting Item, No Sales"""
"""51-Relisting item"""
"""50-Supplier inform item OOS un…"


In [ ]:
# Filter and retrieve values directly from 'Week-Order Remark' when 'MOQ Delivery Conditions' matches
MOQsummary = (
    NoOrder_df
    .group_by("ItemNo")
    .agg([
        pl.when(pl.col("MOQ Delivery Conditions").str.contains(r"(?i)Yes- Not meet MOQ Delivery", literal=False))  # Match the condition
        .then(pl.col("Week-Order Remark"))  # Retrieve 'Week-Order Remark'
        .filter(pl.col("Week-Order Remark").is_not_null())  # Ensure valid values
        .implode()  # Combine into a list
        .alias("MOQ Delivery Indicator Summary")  # Name the aggregated column
    ])
)

# Join the summary back to the original DataFrame
NoOrder_df = NoOrder_df.join(MOQsummary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\772903865.py:15: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
NoOrder_df.filter(pl.col('MOQ Delivery Indicators')!='No- Not MOQ Delivery').sample(2)

Week - EDD,Week,Division,ItemNo,Item Name,RQ Status,Delivery based on truck condition 1. Full truck 2. X Full truck 3. Pallet,Revised Rounded Order Quantity,Order indicator,This Week Order Remark,Planner Remark,Week-Order Remark,Existing Reorder Y,Relisting Item Indicator,Relisting Item Listing,"Control store rate >2.0, total SOH ES >1k and EP >60PCS",Slow Moving Item,Slow Moving Item Indicator,Period Slow Moving,Item To Clear Indicator,Balance To Clear,MOQ Delivery Conditions,MOQ Delivery Indicators,WH ADJ qty (Last week),week1triggersoft,WH ADJ qty Summary,Triggered Qty Summary,WH Adjustment (Write Off),WH Adjustment Category,Adjustment Weekly Summary,Weeks_No_Order,No Order Indicator,Week-No Order Remark,MOQ Delivery Indicator Summary
str,str,str,str,str,str,str,i64,str,str,str,str,str,str,list[list[str]],str,str,str,str,str,str,str,str,i64,i64,list[list[str]],list[list[str]],str,str,list[list[str]],list[str],str,list[str],list[list[str]]
"""52-2""","""52""","""GMS""","""64567029""","""CASA 2 TIER RACK 128 39.8X22X6…","""Active""","""No Delivery Conditions""",0,"""No Order""","""All Order + DC SOH > Demand""","""buffer for CNY closure""","""52 - All Order + DC SOH > Dema…","""Existing Reorder Y""","""No- Not Relisting Item""",null,null,"""52 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 52""","""Yes- Not meet MOQ Delivery in …",0,60,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-36"", ""50-84"", … ""52-60""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]","[""51"", ""52""]","""No order on weeks 51, 52""","[""51-Does not meet one carton requirement"", ""52-All Order + DC SOH > Demand""]","[[""51 - Does not meet one carton requirement"", null]]"
"""52-7""","""52""","""SOFTLINE""","""31015069""","""8719 KCS3988MC HEAT CARTOON BR…","""Active""","""No Delivery Conditions""",0,"""No Order""","""All Order + DC SOH > Demand""",null,"""52 - All Order + DC SOH > Dema…","""Existing Reorder Y""","""No- Not Relisting Item""",null,null,"""52 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""Yes- Balance To Clear Item""","""Balance to clear in weeks 1, 5…","""No- Not MOQ Delivery in 52""","""Yes- Not meet MOQ Delivery in …",0,1020,"[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-870"", ""50-1310"", … ""52-1020""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]","[""50"", ""52""]","""No order on weeks 50, 52""","[""50-Does not meet one carton requirement"", ""52-All Order + DC SOH > Demand""]","[[""50 - Does not meet one carton requirement"", null]]"


In [ ]:
# Select columns ItemNo, Weeks_No_Order, No Order Indicator, Week-No Order Remark, MOQ Delivery Indicator Summary
NoOrder_summary = NoOrder_df.select([
    "ItemNo",
    "Weeks_No_Order",
    "No Order Indicator",
    "Week-No Order Remark",
    "MOQ Delivery Indicator Summary"
])

In [ ]:
# Keep ItemNo in NoOrder_df as unique, so that later can join merge with AnalysisSOH
NoOrder_summary = NoOrder_summary.unique(subset="ItemNo")

# Merge NoOrder_summary with AnalysisSOH
AnalysisSOH_df = AnalysisSOH_df.join(NoOrder_summary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\386266311.py:5: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
AnalysisSOH_df.sample(2)

Division,ABC SKUs,ItemNo,ItemName,Vendor Code,Vendor Name,WarehouseID,Local/Oversea,RQStatus,ProductEntity,BuyerGroup,Department,Category,HouseBrandIndicator,RawMaterialIndicator,StockReorderIndicator,Last RC Date,1st TR In Date,ADS Qty,RL91 Sales Qty,TR date New Item,ADS Qty New Item,WarehouseSOH,Stock cover days,SOH_Condition,StockCoverDays_Condition,OOS_Status,Item Maintenance,Relisting Item Indicator,Relisting Item Listing,Slow Moving Item,Slow Moving Item Indicator,Period Slow Moving,Item To Clear Indicator,Balance To Clear,MOQ Delivery Conditions,MOQ Delivery Indicators,WH ADJ qty Summary,Triggered Qty Summary,WH Adjustment (Write Off),WH Adjustment Category,Adjustment Weekly Summary,Weeks_No_Order,No Order Indicator,Week-No Order Remark,MOQ Delivery Indicator Summary
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,i64,i64,i64,f64,i64,f64,bool,bool,str,str,str,list[list[str]],str,str,str,str,str,str,str,list[list[str]],list[list[str]],str,str,list[list[str]],list[str],str,list[str],list[list[str]]
"""FOOD""","""C""","""14099569""","""FARM FRESH UHT YOGURT - MANGO …","""TPREI0002""","""Reitozo Pacific Sdn Bhd""","""WJI""","""LOCAL""","""Active""","""Eco-Shop""","""EPS-L""","""GROCERY""","""READY TO DRINK (RTD)""","""N""","""N""","""Y""",2024-12-30,2023-07-20,147,13367,null,147.0,352,0.026334,true,true,"""OOS""","""""","""No- Not Relisting Item""",null,"""1 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 1""","""No- Not MOQ Delivery""","[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-5536"", ""50-5728"", … ""52-4192""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]","[""1"", ""52""]","""No order on weeks 1, 52""","[""1-Supplier Closure"", ""52-All Order + DC SOH > Demand""]","[[null, null]]"
"""FOOD""","""B""","""11495002""","""ECOSAFA 100% MINYAK MASAK SAYU…","""TPABL0001""","""Able Perfect Sdn Bhd""","""WJI""","""LOCAL""","""Active""","""Eco-Shop""","""A""","""GROCERY""","""OIL""","""N""","""N""","""Y""",2024-12-23,2023-06-12,128,11682,null,128.0,-48,-0.004109,true,true,"""OOS""","""""","""No- Not Relisting Item""",null,"""1 - nan""","""No-Not Slow Moving Item""","""Not Slow Moving Item""","""No- Not Balance To Clear Item""","""""","""No- Not MOQ Delivery in 1""","""No- Not MOQ Delivery""","[[""1-0"", ""50-0"", … ""52-0""]]","[[""1-912"", ""50-1920"", … ""52-1632""]]","""0.0%""","""<30% Adjustment not affect OOS""","[[""1-<30% Adjustment not affect OOS"", ""50-<30% Adjustment not affect OOS"", … ""52-<30% Adjustment not affect OOS""]]","[""51"", ""52""]","""No order on weeks 51, 52""","[""51-All Order + DC SOH > Demand"", ""52-All Order + DC SOH > Demand""]","[[null, null]]"


## LATEST ADJUSTMENT FOR LATEST WEEK

In [ ]:
# Check only for adjustment that (-) which means got write off, minus stock
Adjustment_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Inv02Adjustment"

# Find or get the latest file inside Adjustment folder
latest_adj = max(glob.glob(Adjustment_path + "/*.xlsx"), key=os.path.getctime)
print(f"The latest adjustment file: {latest_adj}")

# Read using polars
Adj_df = pl.read_excel(latest_adj,
                       read_options={'schema_overrides': {'Item No': pl.Utf8}})\
                      .rename({"Item No": "ItemNo"})\
                      .filter(pl.col("ItemNo").is_not_null(),
                              pl.col('Qty')<0)\
                      .with_columns(pl.col('Qty').abs().alias('Qty'))

# Combine
Adj_df = Adj_df.join(minmax4weeks.with_columns(minmax4weeks['Week'].cast(pl.Int64)),
                         on=['ItemNo','Week'], how='left')

The latest adjustment file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Inv02Adjustment\20250106.xlsx


C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\4269561239.py:17: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Calculate "Latest week WH Adj %"
Adj_df = Adj_df.with_columns(
    pl.when(
        (pl.col("Qty").is_not_null()) &
        (pl.col("week1triggersoft").is_not_null()) &
        (pl.col("week1triggersoft") != 0)
    )
    .then((pl.col("Qty") / pl.col("week1triggersoft") * 100).round(2).cast(pl.Utf8) + "%")
    .otherwise(pl.lit("No Data"))
    .alias("Latest week WH Adj %")
)

# Create "Latest Adj Category Issues"
Adj_df = Adj_df.with_columns(
    pl.when((pl.col("Qty") == 0) & (pl.col("week1triggersoft") == 0))
    .then(pl.lit("<30% Adjustment not affect OOS"))
    .when((pl.col("Qty").is_not_null()) &
          (pl.col("Qty") >= (pl.col("week1triggersoft") * 0.3)))
    .then(pl.lit(">30% Adjustment Affect OOS"))
    .when((pl.col("Qty").is_not_null()) &
          (pl.col("Qty") < (pl.col("week1triggersoft") * 0.3)))
    .then(pl.lit("<30% Adjustment not affect OOS"))
    .otherwise(pl.lit("No Adjustment"))
    .alias("Latest Adj Category Issues")
)

In [ ]:
# Select ItemNo, Latest week WH Adj %, Latest Adj Category Issues
Adj_df = Adj_df.select([
    "ItemNo",
    "Latest week WH Adj %",
    "Latest Adj Category Issues"
])

In [ ]:
# Merge Adj_df with AnalysisSOH
AnalysisSOH_df = AnalysisSOH_df.join(Adj_df, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\1932015591.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# @title Master file for Food and NonFood
# Check for special cases where the supplier have lead time longer than <= 90 days, =>30. Need to run for item which lead time >30 days [Output: Long lead time: {lt_days}]
nonfood_master = fr"{username}\OneDrive - Eco-Shop\Supply Chain x Merchandising Department\Ordering\Ordering Master - Non-Food\Supply Chain order method of Master List - Non-Food,GMS,Softline.xlsx"
food_master = fr"{username}\OneDrive - Eco-Shop\Supply Chain x Merchandising Department\Ordering\Ordering Master - Food\Master List - Local Food-new.xlsx"

# Read the Excel file with Polars
try:
    nf_df = pd.read_excel(
        nonfood_master,
        sheet_name='Master List',  # Replace with your actual sheet name
        header=4, # Row index 3 corresponds to row 4 in Excel
        usecols=['Item No','Lead time/Order in Advance\n(count by days)'],
        dtype={'Item No': str, 'Lead time/Order in Advance\n(count by days)': str}

    )
except Exception as e:
    print(f"Error reading Non-Food Master List: {e}")

# Read the Excel file with Polars
try:
    f_df = pd.read_excel(
        food_master,
        sheet_name='FOOD',  # Replace with your actual sheet name
        header=3,  # Row index 3 corresponds to row 4 in Excel
        usecols=['Item No','Lead time/Order in Advance\n(count by days)'],
        dtype={'Item No': str, 'Lead time/Order in Advance\n(count by days)': str}
    )
except Exception as e:
    print(f"Error reading Non-Food Master List: {e}")

# Combine both dataframe
master_df = pd.concat([nf_df, f_df], ignore_index=True)

# Convert to polars
master_df = pl.from_pandas(master_df)

# rename Item No to ItemNo
master_df = master_df.rename({"Item No": "ItemNo"})

# Supplier SL & FL | Backhaul fulfillment

In [ ]:
# Read purchase SL and Fulfillment file
slfl_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Pur04SupplierSLFL"

# Find or get the latest file inside SupplierFulfillment folder
slfl = max(glob.glob(slfl_path + "/*.xlsx"), key=os.path.getctime)

print(f"The latest SLFL file: {slfl}")

# Read the file using polars
slfl_df = (
    pl.read_excel(slfl) # Changed read_excel_io to read_excel
    .select(
        [
            pl.col("Item No").cast(pl.Utf8),
            pl.col("Vendor Code"),
            pl.col("Vendor Name"),
            pl.col("Order Qty").cast(pl.Int64),
            pl.col("Receive Qty").cast(pl.Int64),
            pl.col("Service Level %"),
            pl.col("Fulfillment %"),
        ]
    )
    .filter(pl.col("Item No").is_not_null())  # Filter out rows where 'Item No' is null
    .with_columns(
        [
            (pl.col("Service Level %") * 100).round(2).alias("Service Level %"),
            (pl.col("Fulfillment %") * 100).round(2).alias("Fulfillment %"),
        ]
    )
    .rename({"Item No": "ItemNo"})  # Rename "Item No" to "ItemNo"
)


# Pair it up with backhaul
backhaul_path = fr"{username}\OneDrive - Eco-Shop\Inventory\Sakura\Backhaul\00 Master Own Transport Data 20250106.xlsx"

backhaul_df = (
    pl.read_excel(
        backhaul_path,
        sheet_name="Master By Item",
    )
    .select(
        [
            pl.col("Vendor Code").cast(pl.Utf8),  # Cast "Vendor Code" to Utf8
            pl.col("Vendor Name"),
            pl.col("Item No").cast(pl.Utf8),  # Cast "Item No" to Utf8
            pl.col("Item Name"),
            pl.col("Vendor Own Transport Indicator"),
            pl.col("Warehouse Group"),
            pl.col("Area"),
        ]
    )
    .rename({"Item No": "ItemNo"})  # Rename "Item No" to "ItemNo

)

# Complete vendor own transport from Qliksense check-up
ownTransport_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\VendorOwnTransportCheck.xlsx"

ownTransport_df = (
    pl.read_excel(ownTransport_path,
                  schema_overrides={"SOH Qty": pl.Float64})
    .select(
        [
            pl.col("Item No").cast(pl.Utf8),  # Cast "Item No" to Utf8
            pl.col("Vendor Own Transport Indicator"),
        ]
    )
    .rename({"Item No": "ItemNo"})  # Rename "Item No" to "Item
    .filter(pl.col("ItemNo").is_not_null())  # Filter out rows where 'Item No' is null
)

The latest SLFL file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Pur04SupplierSLFL\06012025xlsx.xlsx


In [ ]:
# Merge Vendor Own Transport Indicator directly into slfl_df
slfl_df = slfl_df.join(ownTransport_df.select(['ItemNo', 'Vendor Own Transport Indicator']), on='ItemNo', how='left',coalesce=True)

In [ ]:
# Add new columns: Service Level Category and Fulfillment Level Category
slfl_df = slfl_df.with_columns([
    # Service Level Category
    pl.when(pl.col('Vendor Own Transport Indicator') == 'N')
      .then(
          pl.when(pl.col('Service Level %') < 80)
            .then(pl.lit('<80 % Supplier Service Level'))
            .otherwise(pl.lit('>80 % Supplier Service Level'))
      )
      .otherwise(pl.lit('Backhaul transport pickup'))
      .alias('Service Level Category'),

    # Fulfillment Level Category
    pl.when(pl.col('Vendor Own Transport Indicator') == 'N')
      .then(
          pl.when(pl.col('Fulfillment %') < 80)
            .then(pl.lit('<80 % Supplier Fulfillment'))
            .otherwise(pl.lit('>80 % Supplier Fulfillment'))
      )
      .otherwise(
          pl.when(pl.col('Fulfillment %') < 80)
            .then(pl.lit('<80 % Backhaul-Supplier Fulfillment'))
            .otherwise(pl.lit('>80 % Backhaul-Supplier Fulfillment'))
      )
      .alias('Fulfillment Level Category')
])

In [ ]:
# Drop columns Order Qty	Receive Qty
slfl_df = slfl_df.drop(['Order Qty', 'Receive Qty'])

In [ ]:
display(slfl_df)

ItemNo,Vendor Code,Vendor Name,Service Level %,Fulfillment %,Vendor Own Transport Indicator,Service Level Category,Fulfillment Level Category
str,str,str,f64,f64,str,str,str
"""10010158""","""TPINF0004""","""Infinite Multi Resources Sdn B…",100.0,100.0,"""N""",""">80 % Supplier Service Level""",""">80 % Supplier Fulfillment"""
"""10010165""","""TPINF0004""","""Infinite Multi Resources Sdn B…",100.0,100.0,"""N""",""">80 % Supplier Service Level""",""">80 % Supplier Fulfillment"""
"""10010184""","""TPIND0003""","""Indadi Utama (M) Sdn Bhd""",100.0,100.0,"""N""",""">80 % Supplier Service Level""",""">80 % Supplier Fulfillment"""
"""10010185""","""TPIND0003""","""Indadi Utama (M) Sdn Bhd""",100.0,100.0,"""N""",""">80 % Supplier Service Level""",""">80 % Supplier Fulfillment"""
"""10010263""","""TPMAY0001""","""Mayora Malaysia Sdn Bhd""",23.87,71.6,"""Y""","""Backhaul transport pickup""","""<80 % Backhaul-Supplier Fulfil…"
…,…,…,…,…,…,…,…
"""55065028""","""TPNEW0010""","""New Nice Trading (M) Sdn Bhd""",100.0,100.0,"""N""",""">80 % Supplier Service Level""",""">80 % Supplier Fulfillment"""
"""55065032""","""TPMIN0003""","""Minna World Sdn Bhd""",100.0,100.0,"""Y""","""Backhaul transport pickup""",""">80 % Backhaul-Supplier Fulfil…"
"""55065033""","""TPMIN0003""","""Minna World Sdn Bhd""",100.0,100.0,"""Y""","""Backhaul transport pickup""",""">80 % Backhaul-Supplier Fulfil…"


In [ ]:
# Only choose columns ItemNo, Service Level %, Fulfillment %, Vendor Own Transport Indicator, Service Level Category, Fulfillment Level Category
slfl_summary = slfl_df.select([
    "ItemNo",
    "Service Level %",
    "Fulfillment %",
    "Vendor Own Transport Indicator",
    "Service Level Category",
    "Fulfillment Level Category"
])

In [ ]:
# Merge slfil_summary with AnalysisSOH_df
AnalysisSOH_df = AnalysisSOH_df.join(slfl_summary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\1944593727.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



## ShipOut and Receive Data and Trigger Qty


In [ ]:
# @title Shipout file
# Read the ship out vs triggered qty
shipOut_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Inv02ShipOut"

# Find or get the latest file inside SupplierFulfillment folder
shipOut = max(glob.glob(shipOut_path + "/*.xlsx"), key=os.path.getctime)
print(f"The latest ship out file: {shipOut}")

# Qty column data to be absolute
shipOut_df = (
    pl.read_excel(shipOut)
    .with_columns([
        pl.col("Item No").cast(pl.Utf8),  # Cast 'Item No' to string (Utf8 in Polars)
        pl.col("Qty").abs()  # Make 'Qty' column absolute
    ])
    .rename({"Item No": "ItemNo"})  # Rename 'Item No' to 'ItemNo'
)

The latest ship out file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Inv02ShipOut\20240106 Shipment.xlsx


In [ ]:
# Group by "Item No" and aggregate total quantity per week with a single operation
shipOut_summary = (
    shipOut_df
    .with_columns([
        pl.col("Week").cast(pl.Utf8),  # Ensure Week is a string
        pl.col("Qty").cast(pl.Int64)   # Ensure Qty is an integer for summing
    ])
    .group_by(["ItemNo", "Week"])  # Group by Item No and Week
    .agg(pl.col("Qty").sum().alias("Total ShipOut Qty"))  # Aggregate Qty by Week for each Item
    .with_columns(
        (pl.col("Week") + "-" + pl.col("Total ShipOut Qty").cast(pl.Utf8)).alias("Week-Qty")
    )
    .group_by("ItemNo")
    .agg(
        pl.col("Week-Qty").alias("Week-Qty"),  # Collect the Week-Qty pairs as a list
        pl.col("Total ShipOut Qty").sum().alias("Total ShipOut Qty")
    )
    .with_columns([
        pl.col("Week-Qty")
        .apply(lambda x: sorted(x, key=lambda item: int(item.split("-")[0])))  # Sort by week number
        .alias("Week-ShipOut Qty")  # Overwrite the column with the sorted list
    ])
)

# Display the resulting DataFrame for only select column ItemNo and Week-ShipOut Qty
shipOut_summary=shipOut_summary.select(["ItemNo", "Week-ShipOut Qty"])

display(shipOut_summary)

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\293727417.py:20: DeprecationWarning:

`apply` is deprecated. It has been renamed to `map_elements`.

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\293727417.py:18: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ItemNo,Week-ShipOut Qty
str,list[str]
"""65110135""","[""50-516"", ""51-120"", ""52-36""]"
"""33030146""","[""1-384"", ""50-60"", … ""52-2028""]"
"""74490047""","[""1-1080"", ""50-420"", … ""52-720""]"
"""50140930""","[""1-180"", ""50-156"", … ""52-432""]"
"""64180362""","[""1-96"", ""50-108"", … ""52-300""]"
…,…
"""72170875""","[""1-216"", ""51-12"", ""52-120""]"
"""65027205""","[""50-36"", ""51-12""]"
"""63260186""","[""1-192"", ""50-144"", … ""52-216""]"


In [ ]:
# Merge shipOut_summary with AnalysisSOH_df
AnalysisSOH_df = AnalysisSOH_df.join(shipOut_summary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\3500259065.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Read receive qty file
receive_path = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Pur04-4WeeksReceive"

# Find or get the latest file inside SupplierFulfillment folder
latest_rc = max(glob.glob(receive_path + "/*.xlsx"), key=os.path.getctime)
print(f"The latest ship out file: {latest_rc}")

# Qty column data to be absolute
receive_df = (
    pl.read_excel(latest_rc)
    .with_columns([
        pl.col("Item No").cast(pl.Utf8),  # Cast 'Item No' to string (Utf8 in Polars)
        pl.col("Order Qty").cast(pl.Int64),
        pl.col("Receive Qty").cast(pl.Int64)
    ])
    .rename({"Item No": "ItemNo"})  # Rename 'Item No' to 'ItemNo'
    .filter(pl.col("ItemNo").is_not_null())  # Filter out rows where 'Item No' is null
)

The latest ship out file: C:\Users\arfahanis\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\Pur04-4WeeksReceive\06012025.xlsx


In [ ]:
receive_df.sample(2)

Division,ItemNo,Raw Material Indicator,Stock Reorder Indicator,Local/Oversea,Vendor Code,Vendor Name,Week,Order Qty,Receive Qty,Service Level %,Fulfillment %
str,str,str,str,str,str,str,i64,i64,i64,f64,f64
"""GMS""","""65085203""","""N""","""Y""","""LOCAL""","""TPSHE0008""","""Sheng Wang Industries Sdn Bhd""",1,23424,12480,0.532787,0.532787
"""GMS""","""64253025""","""N""","""Y""","""LOCAL""","""TPDRA0002""","""Dragonpak Industries (M) Sdn B…",51,480,480,0.0,1.0


In [ ]:
receive_summary = (
    receive_df
    .group_by(["ItemNo", "Week"])
    .agg([
        pl.col("Order Qty").sum().alias("Total Order Qty"),      # Sum Order Qty by Week for each ItemNo
        pl.col("Receive Qty").sum().alias("Total Receive Qty")    # Sum Receive Qty by Week for each ItemNo
    ])
    .with_columns([
        # Concatenate 'Week', 'Order Qty', and 'Receive Qty' to create "Week-OrderQty-ReceiveQty" as a single entry
        (pl.col("Week").cast(pl.Utf8) + "-" +
         pl.col("Total Order Qty").cast(pl.Utf8) + "-" +
         pl.col("Total Receive Qty").cast(pl.Utf8))
        .alias("Week-OrderQty-ReceiveQty"),
        pl.col("Week").cast(pl.Int64).alias("Week_num")  # Cast Week to integer for proper sorting
    ])
    .sort(["ItemNo", "Week_num"])  # Sort by ItemNo and numeric Week
    .group_by("ItemNo")
    .agg([
        # Collect Week-OrderQty-ReceiveQty entries into a list for each ItemNo
        pl.col("Week-OrderQty-ReceiveQty").implode().alias("Week-OrderQty-ReceiveQty"),
        pl.col("Total Order Qty").sum().alias("Total Order Qty"),
        pl.col("Total Receive Qty").sum().alias("Total Receive Qty")
    ])
)

# Receive summary will only select column ItemNo and
receive_summary = receive_summary.select(["ItemNo", "Week-OrderQty-ReceiveQty"])

In [ ]:
display(receive_summary)

ItemNo,Week-OrderQty-ReceiveQty
str,list[list[str]]
"""99099055""","[[""50-3-0"", ""51-1-0""]]"
"""67017029""","[[""51-75-0""]]"
"""65205044""","[[""1-1776-888"", ""51-708-708"", ""52-648-648""]]"
"""99218233""","[[""51-25-0""]]"
"""10155061""","[[""1-1800-1800"", ""51-1296-1296"", ""52-972-972""]]"
…,…
"""10085018""","[[""1-4050-4050"", ""51-2460-2460""]]"
"""10053058""","[[""1-320-0""]]"
"""31015027""","[[""1-500-500"", ""50-900-900"", … ""52-250-250""]]"


In [ ]:
# Merge receive_summary with AnalysisSOH_df
AnalysisSOH_df = AnalysisSOH_df.join(receive_summary, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\3426396489.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Find the summary for triggered and ship out and receive
minmax = minmax4weeks.pivot(
    values="week1triggersoft",
    index="ItemNo",
    columns="Week",
)

# Rename columns
minmax = minmax.rename({col: f"week1triggersoft Week{col}" for col in minmax.columns if col != "ItemNo"})

# Ship-out to be transpose
shipOut = shipOut_df.pivot(
    values="Qty",
    index="ItemNo",
    columns="Week",
    aggregate_function="sum"
)

# Rename columns
shipOut = shipOut.rename({col: f"ShipOut Qty Week{col}" for col in shipOut.columns if col != "ItemNo"})

# Receive to be transpose
receive = receive_df.pivot(
    values="Receive Qty",
    index="ItemNo",
    columns="Week",
    aggregate_function="sum"
)

# Rename columns
receive = receive.rename({col: f"Receive Qty Week{col}" for col in receive.columns if col != "ItemNo"})

# Combine minmax and shipOut and receive data frame
mm_so_rc = minmax.join(shipOut, on="ItemNo", how="left", coalesce=True)
mm_so_rc = mm_so_rc.join(receive, on="ItemNo", how="left", coalesce=True)

In [ ]:
# @title Sorted the columns
# Sort columns by type ('week1triggersoft', 'ShipOut', 'Receive') and week number
mm_columns = (
    ['ItemNo'] +  # Always keep 'ItemNo' as the first column
    sorted(
        [col for col in mm_so_rc.columns if "week1triggersoft" in col],
        key=lambda col: int(col.split("Week")[-1])  # Sort by week number
    ) +
    sorted(
        [col for col in mm_so_rc.columns if "ShipOut" in col],
        key=lambda col: int(col.split("Week")[-1])  # Sort by week number
    ) +
    sorted(
        [col for col in mm_so_rc.columns if "Receive" in col],
        key=lambda col: int(col.split("Week")[-1])  # Sort by week number
    )
)

# Reorder the DataFrame based on the sorted columns
mm_so_rc = mm_so_rc.select(mm_columns)

In [ ]:
mm_so_rc.filter(pl.col('ItemNo')=='10010158')

ItemNo,week1triggersoft Week1,week1triggersoft Week50,week1triggersoft Week51,week1triggersoft Week52,ShipOut Qty Week1,ShipOut Qty Week50,ShipOut Qty Week51,ShipOut Qty Week52,Receive Qty Week1,Receive Qty Week50,Receive Qty Week51,Receive Qty Week52
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""10010158""",6360,5232,6840,7008,4488,2640,11088,4344,6744,null,4032,6456


In [ ]:
# @title Merge mm_so_rc with master_df

# Merge mm_so_rc with master_df
mm_so_rc = mm_so_rc.join(master_df, on="ItemNo", how="left", coalesce=True)

# Calculate Lead time (Week)
mm_so_rc = mm_so_rc.with_columns(
    (pl.col('Lead time/Order in Advance\n(count by days)').cast(pl.Float64) / 7).ceil().alias('Lead time (Week)')
)

In [ ]:
# Define a function to get the correct week number and year considering the wrap around
def get_correct_week(year, week):
    if week <= 0:
        year -= 1
        week += 52  # Assuming no leap week; adjust if using ISO weeks and leap week might occur
    return year, week

# Define the week numbers dynamically (e.g., last 4 weeks)
dynamic_weeks = [(current_year, current_week_number - i) for i in range(4, 0, -1)]
corrected_weeks = [get_correct_week(year, week) for year, week in dynamic_weeks]

# Define dynamic column names for week1triggersoft, ShipOut Qty, and Receive Qty
trigger_cols = [f"week1triggersoft Week{week}" for year, week in corrected_weeks]
shipout_cols = [f"ShipOut Qty Week{week}" for year, week in corrected_weeks]
receive_cols = [f"Receive Qty Week{week}" for year, week in corrected_weeks]

# Calculate Final Trigger Qty dynamically
mm_so_rc = mm_so_rc.with_columns(
    pl.reduce(
        lambda a, b: a + b,
        [pl.col(col) * (pl.lit(week_index + 1) <= pl.col("Lead time (Week)")) for week_index, col in enumerate(trigger_cols)]
    ).alias('Final Trigger Qty')
)

# Calculate Final Ship Out Qty dynamically
mm_so_rc = mm_so_rc.with_columns(
    pl.reduce(
        lambda a, b: a + b,
        [pl.col(col) * (pl.lit(week_index + 1) <= pl.col("Lead time (Week)")) for week_index, col in enumerate(shipout_cols)]
    ).alias('Final Ship Out Qty')
)

# Calculate Total Receive Qty dynamically
mm_so_rc = mm_so_rc.with_columns(
    pl.reduce(
        lambda a, b: a + b,
        [pl.col(col) for col in receive_cols]
    ).alias('Total Receive Qty')
)

# Replace null values with 0
mm_so_rc = mm_so_rc.fill_null(0)

# Categorize based on Final Ship Out and Trigger Qty
mm_so_rc = mm_so_rc.with_columns(
    pl.when(pl.col("Final Ship Out Qty") > pl.col("Final Trigger Qty"))
      .then(pl.lit("Final Ship Out > Final Trigger"))
      .when(pl.col("Final Ship Out Qty") < pl.col("Final Trigger Qty"))
      .then(pl.lit("Final Ship Out < Final Trigger"))
      .otherwise(pl.lit("Final Ship Out = Final Trigger"))
      .alias("Ship Out vs Trigger Category")
)

# Categorize based on Final Ship Out Qty and Total Receive Qty
mm_so_rc = mm_so_rc.with_columns(
    pl.when(pl.col("Final Ship Out Qty") > pl.col("Total Receive Qty"))
      .then(pl.lit("Final Ship Out > Total Receive"))
      .when(pl.col("Final Ship Out Qty") < pl.col("Total Receive Qty"))
      .then(pl.lit("Final Ship Out < Total Receive"))
      .otherwise(pl.lit("Final Ship Out = Total Receive"))
      .alias("Ship Out vs Receive Category")
)

In [ ]:
mm_so_rc.sample(2)

ItemNo,week1triggersoft Week1,week1triggersoft Week50,week1triggersoft Week51,week1triggersoft Week52,ShipOut Qty Week1,ShipOut Qty Week50,ShipOut Qty Week51,ShipOut Qty Week52,Receive Qty Week1,Receive Qty Week50,Receive Qty Week51,Receive Qty Week52,Lead time/Order in Advance (count by days),Lead time (Week),Final Trigger Qty,Final Ship Out Qty,Total Receive Qty,Ship Out vs Trigger Category,Ship Out vs Receive Category
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,i64,str,str
"""32337005""",216,192,216,216,0,0,0,0,0,0,0,0,"""14""",2.0,408,0,0,"""Final Ship Out < Final Trigger""","""Final Ship Out = Total Receive"""
"""64010047""",228,2004,2052,228,0,0,0,0,0,0,0,0,null,0.0,0,0,0,"""Final Ship Out = Final Trigger""","""Final Ship Out = Total Receive"""


In [ ]:
# Save to excel
mm_so_rc.write_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\mm_so_rc")

In [ ]:
# Rearrange those columns which ItemNo + Lead time/Order in Advance (count by days) + Lead time (Week) + then those all columns
columns_order = (
    ["ItemNo", "Lead time/Order in Advance\n(count by days)", "Lead time (Week)"] +
    [col for col in mm_so_rc.columns if col not in ["ItemNo", "Lead time/Order in Advance\n(count by days)", "Lead time (Week)"]]
)

# Rearrange the columns
mm_so_rc = mm_so_rc.select(columns_order)

In [ ]:
# Merge mm_so_rc with AnalysisSOH
AnalysisSOH_df = AnalysisSOH_df.join(mm_so_rc, on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\1189552136.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Add new column which is "Long Lead Time". Which is detect if Lead time/Order in Advance\n(count by days)" > 30, categorize it as Long LT affect OOS else Normal LT
AnalysisSOH_df = AnalysisSOH_df.with_columns(
    pl.when(pl.col("Lead time/Order in Advance\n(count by days)").cast(pl.Int32) > 30)  # when condition, explicitly cast to Int32
    .then(pl.lit("Long LT"))  # if true, assign "Long LT"
    .otherwise(pl.lit("Normal LT")) # if false, assign "Normal LT"
    .alias("Long Lead Time")  # rename the new column
)

In [ ]:
avg_LT = fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Ordering File\Master File\Supplier Actual Lead Time\Supplier Lead Time.xlsx"
leadtime_setting = pl.read_excel(
                      avg_LT,
                      sheet_name="By SKU",
                      read_options={
                        'schema_overrides': {'Order Qty': pl.Float64},  # Explicitly set 'Order Qty' to Float64
                        'infer_schema_length': 10000  # Increase schema inference length if needed
                    }
                      # seldct column ItemNo and Supplier Lead Time & RL 91 Actual Lead Time
                  ).select([
                        pl.col("ItemNo").cast(pl.Utf8),  # Cast 'Item No' to string (Utf8 in Polars)
                        pl.col("Supplier Lead Time"),
                        pl.col("RL91 Actual Lead Time")
                      ])

In [ ]:
# Compare each ItemNo which if RL91 Actual Lead Time > Supplier Lead Time, indicate as Avergae LT > Supplier LT, else Average LT < Supplier LT
leadtime_setting = leadtime_setting.with_columns(
    pl.when(pl.col("RL91 Actual Lead Time").cast(pl.Float64) > pl.col("Supplier Lead Time").cast(pl.Float64))
    .then(pl.lit("Average LT > Supplier LT"))
    .otherwise(pl.lit("Average LT < Supplier LT"))
    .alias("Leadtime setting different")
)

In [ ]:
leadtime_setting

ItemNo,Supplier Lead Time,RL91 Actual Lead Time,Leadtime setting different
str,i64,f64,str
"""72285032""",30,13.666667,"""Average LT < Supplier LT"""
"""72285050""",30,14.714286,"""Average LT < Supplier LT"""
"""72285028""",30,13.571429,"""Average LT < Supplier LT"""
"""72285035""",30,13.285714,"""Average LT < Supplier LT"""
"""14085023""",90,73.926829,"""Average LT < Supplier LT"""
…,…,…,…
"""32133034""",14,13.0,"""Average LT < Supplier LT"""
"""32013095""",14,null,"""Average LT < Supplier LT"""
"""32013090""",14,7.0,"""Average LT < Supplier LT"""


In [ ]:
# Combine only column Leadtime setting different from leadtime_setting dataframe with AnalysisSOH_df
AnalysisSOH_df = AnalysisSOH_df.join(leadtime_setting.select(["ItemNo","Leadtime setting different"]), on="ItemNo", how="left")

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\4266188542.py:2: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [ ]:
# Save AnalysisSOH to excel
AnalysisSOH_df.write_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\Last_AnalysisSOH")

In [ ]:
# Dynamically identify week-related columns
week_triggersoft_columns = [col for col in AnalysisSOH_df.columns if col.startswith('week1triggersoft')]
shipout_qty_columns = [col for col in AnalysisSOH_df.columns if col.startswith('ShipOut Qty')]
receive_qty_columns = [col for col in AnalysisSOH_df.columns if col.startswith('Receive Qty')]

# Combine all week-related columns in the desired order
week_columns = week_triggersoft_columns + shipout_qty_columns + receive_qty_columns

# Define static columns before "Lead time (Week)"
static_columns_before = [
    'Division',
    'ABC SKUs',
    'ItemNo',
    'ItemName',
    'Vendor Code',
    'Vendor Name',
    'WarehouseID',
    'Local/Oversea',
    'RQStatus',
    'ProductEntity',
    'BuyerGroup',
    'Department',
    'Category',
    'HouseBrandIndicator',
    'RawMaterialIndicator',
    'StockReorderIndicator',
    'Last RC Date',
    '1st TR In Date',
    'ADS Qty',
    'RL91 Sales Qty',
    'TR date New Item',
    'ADS Qty New Item',
    'WarehouseSOH',
    'Stock cover days',
    'SOH_Condition',
    'StockCoverDays_Condition',
    'OOS_Status',
    'Item Maintenance',
    'Relisting Item Indicator',
    'Relisting Item Listing',
    'Slow Moving Item',
    'Slow Moving Item Indicator',
    'Period Slow Moving',
    'Item To Clear Indicator',
    'Balance To Clear',
    'MOQ Delivery Conditions',
    'MOQ Delivery Indicators',
    'MOQ Delivery Indicator Summary',
    'WH ADJ qty Summary',
    'Triggered Qty Summary',
    'WH Adjustment (Write Off)',
    'WH Adjustment Category',
    'Adjustment Weekly Summary',
    'Latest week WH Adj %',
    'Latest Adj Category Issues',
    'Weeks_No_Order',
    'No Order Indicator',
    'Week-No Order Remark',
    'Service Level %',
    'Fulfillment %',
    'Vendor Own Transport Indicator',
    'Service Level Category',
    'Fulfillment Level Category',
    'Week-ShipOut Qty',
    'Week-OrderQty-ReceiveQty',
    'Lead time/Order in Advance\n(count by days)',
    'Lead time (Week)'
]

# Define static columns after the week-related columns
static_columns_after = [
    'Final Trigger Qty',
    'Final Ship Out Qty',
    'Total Receive Qty',
    'Ship Out vs Trigger Category',
    'Ship Out vs Receive Category',
    'Long Lead Time',
    'Leadtime setting different'
]

# Combine static columns and week columns in the correct order
final_column_order = static_columns_before + week_columns + static_columns_after

# Rearrange the columns in the DataFrame
AnalysisSOH_df = AnalysisSOH_df[final_column_order]

# Verify the column order
print(AnalysisSOH_df.columns)

['Division', 'ABC SKUs', 'ItemNo', 'ItemName', 'Vendor Code', 'Vendor Name', 'WarehouseID', 'Local/Oversea', 'RQStatus', 'ProductEntity', 'BuyerGroup', 'Department', 'Category', 'HouseBrandIndicator', 'RawMaterialIndicator', 'StockReorderIndicator', 'Last RC Date', '1st TR In Date', 'ADS Qty', 'RL91 Sales Qty', 'TR date New Item', 'ADS Qty New Item', 'WarehouseSOH', 'Stock cover days', 'SOH_Condition', 'StockCoverDays_Condition', 'OOS_Status', 'Item Maintenance', 'Relisting Item Indicator', 'Relisting Item Listing', 'Slow Moving Item', 'Slow Moving Item Indicator', 'Period Slow Moving', 'Item To Clear Indicator', 'Balance To Clear', 'MOQ Delivery Conditions', 'MOQ Delivery Indicators', 'MOQ Delivery Indicator Summary', 'WH ADJ qty Summary', 'Triggered Qty Summary', 'WH Adjustment (Write Off)', 'WH Adjustment Category', 'Adjustment Weekly Summary', 'Latest week WH Adj %', 'Latest Adj Category Issues', 'Weeks_No_Order', 'No Order Indicator', 'Week-No Order Remark', 'Service Level %', 'Fu

In [ ]:
# Define the prioritization order
priority_order = [
    'Relisting Item Indicator',
    "Item Maintenance",
    "Slow Moving Item Indicator",
    'MOQ Delivery Indicators',
    "Service Level Category",
    "Fulfillment Level Category",
    "Ship Out vs Trigger Category",
    "Ship Out vs Receive Category",
    "Latest Adj Category Issues",
    "Item To Clear Indicator",
    "Leadtime setting different"
]

# Combine values into 'OOS Summary Reason' while maintaining priority
AnalysisSOH_df = AnalysisSOH_df.with_columns(
    pl.concat_list([pl.col(col).cast(pl.Utf8) for col in priority_order]).alias("OOS Summary Reason")
)

# Ensure 'OOS Summary Reason' is an array column
AnalysisSOH_df = AnalysisSOH_df.with_columns(
    pl.col("OOS Summary Reason").cast(pl.List(pl.Utf8))
)

In [ ]:
# Adjusted remark mapping with corrected keys
remark_mapping = {
    "Yes- Relisting Item": "Item Maintenance",
    "New Item + Eco-Shop": "Item Maintenance",
    "New Item + Eco Plus discontinued": "Item Maintenance",
    "Yes-Slow Moving Item": "Slow Moving Item",
    "Yes- Not meet MOQ Delivery": "MOQ Delivery Condition",  # Corrected this key
    "<80 % Supplier Service Level": "Service Level Category",
    "<80 % Supplier Fulfillment": "Fulfillment Level Category",
    "<80 % Backhaul-Supplier Fulfillment": "Fulfillment Level Category",
    "Final Ship Out > Final Trigger": "Ship Out VS Trigger Category",
    "Final Ship Out > Total Receive": "Ship Out VS Receive Category",
    ">30% Adjustment Affect OOS": "Warehouse Adjustment Issues",
    "Yes- Balance To Clear Item": "Balance To Clear Item",
    "Average LT > Supplier LT": "Long Lead Time"
}

# Flatten the list, map each reason, and return the first match based on priority
def map_nested_reasons(reason_list, mapping):
    if reason_list is None or len(reason_list) == 0:
        return "OTHER POSSIBLE REASONS"
    for reason in reason_list:
        if reason is None or reason.strip() == "":
            continue
        for key, value in mapping.items():
            if reason.startswith(key):
                return value
    return "OTHER POSSIBLE REASONS"

# Apply the mapping function to the nested lists
AnalysisSOH_df = AnalysisSOH_df.with_columns(
    pl.col("OOS Summary Reason")
    .map_elements(lambda reason_list: map_nested_reasons(reason_list, remark_mapping), return_dtype=pl.Utf8)
    .alias("Final OOS Remarks")
)

# Display the result
print(AnalysisSOH_df)

shape: (353, 78)
┌──────────┬──────────┬──────────┬────────────────────┬───┬───────────┬──────────────┬─────────────┬───────────────────┐
│ Division ┆ ABC SKUs ┆ ItemNo   ┆ ItemName           ┆ … ┆ Long Lead ┆ Leadtime     ┆ OOS Summary ┆ Final OOS Remarks │
│ ---      ┆ ---      ┆ ---      ┆ ---                ┆   ┆ Time      ┆ setting      ┆ Reason      ┆ ---               │
│ str      ┆ str      ┆ str      ┆ str                ┆   ┆ ---       ┆ different    ┆ ---         ┆ str               │
│          ┆          ┆          ┆                    ┆   ┆ str       ┆ ---          ┆ list[str]   ┆                   │
│          ┆          ┆          ┆                    ┆   ┆           ┆ str          ┆             ┆                   │
╞══════════╪══════════╪══════════╪════════════════════╪═══╪═══════════╪══════════════╪═════════════╪═══════════════════╡
│ GMS      ┆ C        ┆ 63155073 ┆ CURRY PUFF MOULD   ┆ … ┆ Normal LT ┆ Average LT < ┆ ["No- Not   ┆ Slow Moving Item  │
│          ┆   

In [ ]:
# Final category mapping based on your specified rules
final_category_mapping = {
    "Item Maintenance": "NEW | RELISTING ITEM MAINTENANCE",
    "MOQ Delivery Condition": "ITEM SETTING",
    "Slow Moving Item": "OPERATION ISSUES",
    "Balance To Clear Item": "OPERATION ISSUES",
    "Warehouse Adjustment Issues": "OPERATION ISSUES",
    "Long Lead Time": "ITEM SETTING",
    "Ship Out VS Trigger Category": "OPERATION ISSUES",
    "Ship Out VS Receive Category": "OPERATION ISSUES",
    "Fulfillment Level Category": "SUPPLIER ISSUES",
    "Service Level Category": "SUPPLIER ISSUES",
    "Other Possibel Reasons": "OTHER POSSIBLE REASONS"  # Ensure this is also mapped
}

# Map these to the final categories
AnalysisSOH_df = AnalysisSOH_df.with_columns(
    pl.col("Final OOS Remarks")
    .apply(lambda x: final_category_mapping.get(x, "OTHER POSSIBLE REASONS"))
    .alias("Final OOS Category")
)

# Display the result to verify the mapping
print(AnalysisSOH_df.select(['ItemNo', 'Final OOS Remarks', 'Final OOS Category']))

shape: (353, 3)
┌──────────┬──────────────────────────────┬────────────────────────┐
│ ItemNo   ┆ Final OOS Remarks            ┆ Final OOS Category     │
│ ---      ┆ ---                          ┆ ---                    │
│ str      ┆ str                          ┆ str                    │
╞══════════╪══════════════════════════════╪════════════════════════╡
│ 63155073 ┆ Slow Moving Item             ┆ OPERATION ISSUES       │
│ 64135047 ┆ OTHER POSSIBLE REASONS       ┆ OTHER POSSIBLE REASONS │
│ 63757005 ┆ Service Level Category       ┆ SUPPLIER ISSUES        │
│ 64085015 ┆ Fulfillment Level Category   ┆ SUPPLIER ISSUES        │
│ 64245001 ┆ Ship Out VS Receive Category ┆ OPERATION ISSUES       │
│ …        ┆ …                            ┆ …                      │
│ 10088098 ┆ Fulfillment Level Category   ┆ SUPPLIER ISSUES        │
│ 10089501 ┆ Ship Out VS Trigger Category ┆ OPERATION ISSUES       │
│ 10090003 ┆ Service Level Category       ┆ SUPPLIER ISSUES        │
│ 10095076 ┆ MOQ D

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\2379902922.py:19: DeprecationWarning:

`apply` is deprecated. It has been renamed to `map_elements`.

C:\Users\arfahanis\AppData\Local\Temp\ipykernel_19528\2379902922.py:17: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



In [ ]:
priority_keys = [
    "Yes- Relisting Item",
    "New Item + Eco-Shop",
    "New Item + Eco Plus discontinued",
    "Yes-Slow Moving Item",
    "Yes- Not meet MOQ Delivery",
    "<80 % Supplier Service Level",
    "<80 % Supplier Fulfillment",
    "<80 % Backhaul-Supplier Fulfillment",
    "Final Ship Out > Final Trigger",
    "Final Ship Out > Total Receive",
    ">30% Adjustment Affect OOS",
    "Yes- Balance To Clear Item",
    "Average LT > Supplier LT"
]

# Function to match the first occurrence based on priority keys
def match_first_priority(reason_list):
    if reason_list is None:
        return "Others"
    # Iterate through the priority list and check for presence in the reason list
    for key in priority_keys:
        if any(key in reason for reason in reason_list if reason is not None):
            return key  # Return the key directly when found
    return "Others"

# Apply the function to each row in the DataFrame
AnalysisSOH_df = AnalysisSOH_df.with_columns(
    pl.col("OOS Summary Reason")
    .map_elements(match_first_priority, return_dtype=pl.Utf8)
    .alias("Highlighted OOS Remarks")
)

# Display the updated DataFrame to verify the changes
print(AnalysisSOH_df.select(['OOS Summary Reason', 'Highlighted OOS Remarks']))

shape: (353, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ OOS Summary Reason              ┆ Highlighted OOS Remarks         │
│ ---                             ┆ ---                             │
│ list[str]                       ┆ str                             │
╞═════════════════════════════════╪═════════════════════════════════╡
│ ["No- Not Relisting Item", "",… ┆ Yes-Slow Moving Item            │
│ ["No- Not Relisting Item", "",… ┆ Others                          │
│ ["No- Not Relisting Item", "Ec… ┆ <80 % Supplier Service Level    │
│ ["No- Not Relisting Item", "",… ┆ <80 % Backhaul-Supplier Fulfil… │
│ ["No- Not Relisting Item", "",… ┆ Final Ship Out > Total Receive  │
│ …                               ┆ …                               │
│ ["No- Not Relisting Item", "",… ┆ <80 % Backhaul-Supplier Fulfil… │
│ ["No- Not Relisting Item", "",… ┆ Final Ship Out > Final Trigger  │
│ ["No- Not Relisting Item", "",… ┆ <80 % Supplier Service Level    │
│ ["

In [ ]:
# Dynamically identify week-related columns
week_triggersoft_columns = [col for col in AnalysisSOH_df.columns if col.startswith('week1triggersoft')]
shipout_qty_columns = [col for col in AnalysisSOH_df.columns if col.startswith('ShipOut Qty')]
receive_qty_columns = [col for col in AnalysisSOH_df.columns if col.startswith('Receive Qty')]

# Combine all week-related columns in the desired order
week_columns = week_triggersoft_columns + shipout_qty_columns + receive_qty_columns

# Define the desired final column order based on your specified list
final_column_order = [
    'Division',
    'ItemNo',
    'ItemName',
    'OOS Summary Reason',
    'Highlighted OOS Remarks',
    'Final OOS Remarks',
    'Final OOS Category',
    'ABC SKUs',
    'Vendor Code',
    'Vendor Name',
    'WarehouseID',
    'Local/Oversea',
    'RQStatus',
    'ProductEntity',
    'BuyerGroup',
    'Department',
    'Category',
    'HouseBrandIndicator',
    'RawMaterialIndicator',
    'StockReorderIndicator',
    'Vendor Own Transport Indicator',
    'WarehouseSOH',
    'Stock cover days',
    'SOH_Condition',
    'StockCoverDays_Condition',
    'OOS_Status',
    'Last RC Date',
    '1st TR In Date',
    'ADS Qty',
    'RL91 Sales Qty',
    'TR date New Item',
    'ADS Qty New Item',
    'Item Maintenance',
    'Relisting Item Indicator',
    'Relisting Item Listing',
    'Slow Moving Item',
    'Slow Moving Item Indicator',
    'Period Slow Moving',
    'Item To Clear Indicator',
    'Balance To Clear',
    'Weeks_No_Order',
    'No Order Indicator',
    'Week-No Order Remark',
    'MOQ Delivery Conditions',
    'MOQ Delivery Indicators',
    'MOQ Delivery Indicator Summary',
    'Service Level %',
    'Fulfillment %',
    'Service Level Category',
    'Fulfillment Level Category',
    'Lead time/Order in Advance\n(count by days)',
    'Lead time (Week)',
    'Long Lead Time',
    'Leadtime setting different'
] + week_columns + [
    'Final Trigger Qty',
    'Final Ship Out Qty',
    'Total Receive Qty',
    'Ship Out vs Trigger Category',
    'Ship Out vs Receive Category',
    'WH Adjustment (Write Off)',
    'WH Adjustment Category',
    'Adjustment Weekly Summary',
    'Latest week WH Adj %',
    'Latest Adj Category Issues'
]

# Ensure all expected columns exist in the DataFrame to prevent KeyErrors
missing_cols = [col for col in final_column_order if col not in AnalysisSOH_df.columns]
if missing_cols:
    raise ValueError(f"Missing columns in DataFrame: {missing_cols}")

# Rearrange the columns in the DataFrame according to the final order
AnalysisSOH_df = AnalysisSOH_df.select(final_column_order)

# Print the columns to verify the order
print(AnalysisSOH_df.columns)

['Division', 'ItemNo', 'ItemName', 'OOS Summary Reason', 'Highlighted OOS Remarks', 'Final OOS Remarks', 'Final OOS Category', 'ABC SKUs', 'Vendor Code', 'Vendor Name', 'WarehouseID', 'Local/Oversea', 'RQStatus', 'ProductEntity', 'BuyerGroup', 'Department', 'Category', 'HouseBrandIndicator', 'RawMaterialIndicator', 'StockReorderIndicator', 'Vendor Own Transport Indicator', 'WarehouseSOH', 'Stock cover days', 'SOH_Condition', 'StockCoverDays_Condition', 'OOS_Status', 'Last RC Date', '1st TR In Date', 'ADS Qty', 'RL91 Sales Qty', 'TR date New Item', 'ADS Qty New Item', 'Item Maintenance', 'Relisting Item Indicator', 'Relisting Item Listing', 'Slow Moving Item', 'Slow Moving Item Indicator', 'Period Slow Moving', 'Item To Clear Indicator', 'Balance To Clear', 'Weeks_No_Order', 'No Order Indicator', 'Week-No Order Remark', 'MOQ Delivery Conditions', 'MOQ Delivery Indicators', 'MOQ Delivery Indicator Summary', 'Service Level %', 'Fulfillment %', 'Service Level Category', 'Fulfillment Level 

In [ ]:
# Save AnalysisSOH to excel
AnalysisSOH_df.write_excel(fr"{username}\OneDrive - Eco-Shop\Supply Chain\Ordering Transition (in progress)\Buyer file\Report\WarehouseOOS\OutputOOS\Result\OutputByWeeks\Last_AnalysisSOH_Wk{current_week_number}")